In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')


# See if all the email links have Job Description Text

In [3]:

urls_list = [
    'https://www.indeed.com/viewjob?jk=0203c71ef152dc43&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7909525781251119&adid=382526978&ad=-6NYlbfkN0C8O9VKdOj_1Zh75e9_CvYhSsWVxS1Pvi5WUWhsf4w7FJvt2herunrAVSP821FuTiCRncVK-jw0ey5JdEP7DShfFt_usVJvUUMNEFrG_m1EuDLCkn6V9LRBJYNPIWk2muJ2AYfKSKvgfGW_2nHC2DOzzQVSehcTBCHvNixWgMAQzlRhRO5qoJlrNZpb9fgwfFXWxkpeKkjLWZwkq7RZI0q65y-X90Et5UGI45x_yKBCtSZyl5pdlo4rY6A8HX9d_-6pCMD-yAkncK3Q8w5qlw7GMHz_bgKRSpC0rI8-ClsbyOOBNsbyOCPs09EYb4TRAq-axgNUYEJ2uMqSmrqzZW5_W6CFDhzJaGbpyOJChMyVzZ0L-zARhaTZTtucjWQEYR8%3D&sjdu=QWF4TUFyrHvH7u082A4fa_aRX3M-UVWF5Ox5H4EyFBg&acatk=1g0hp961kpv3g800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=0203c71ef152dc43&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7909525781251119&adid=382526978&ad=-6NYlbfkN0C8O9VKdOj_1Zh75e9_CvYhSsWVxS1Pvi5WUWhsf4w7FJvt2herunrAVSP821FuTiCRncVK-jw0ey5JdEP7DShfFt_usVJvUUMNEFrG_m1EuDLCkn6V9LRBJYNPIWk2muJ2AYfKSKvgfGW_2nHC2DOzzQVSehcTBCHvNixWgMAQzlRhRO5qoJlrNZpb9fgwfFXWxkpeKkjLWZwkq7RZI0q65y-X90Et5UGI45x_yKBCtSZyl5pdlo4rY6A8HX9d_-6pCMD-yAkncK3Q8w5qlw7GMHz_bgKRSpC0rI8-ClsbyOOBNsbyOCPs09EYb4TRAq-axgNUYEJ2uMqSmrqzZW5_W6CFDhzJaGbpyOJChMyVzZ0L-zARhaTZTtucjWQEYR8%3D&sjdu=QWF4TUFyrHvH7u082A4fa_aRX3M-UVWF5Ox5H4EyFBg&acatk=1g0hp961kpv3g800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=0203c71ef152dc43&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7909525781251119&adid=382526978&ad=-6NYlbfkN0C8O9VKdOj_1Zh75e9_CvYhSsWVxS1Pvi5WUWhsf4w7FJvt2herunrAVSP821FuTiCRncVK-jw0ey5JdEP7DShfFt_usVJvUUMNEFrG_m1EuDLCkn6V9LRBJYNPIWk2muJ2AYfKSKvgfGW_2nHC2DOzzQVSehcTBCHvNixWgMAQzlRhRO5qoJlrNZpb9fgwfFXWxkpeKkjLWZwkq7RZI0q65y-X90Et5UGI45x_yKBCtSZyl5pdlo4rY6A8HX9d_-6pCMD-yAkncK3Q8w5qlw7GMHz_bgKRSpC0rI8-ClsbyOOBNsbyOCPs2voCzRgtss6k0wyl8GebK_Ro_ORhNIt56ujnclaOv7keNKKoOcMor9GRNIezCZjXfINHc9O8EVs%3D&sjdu=QWF4TUFyrHvH7u082A4fa_aRX3M-UVWF5Ox5H4EyFBg&acatk=1g0hjl8p4t61o802&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=0203c71ef152dc43&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7909525781251119&adid=382526978&ad=-6NYlbfkN0C8O9VKdOj_1Zh75e9_CvYhSsWVxS1Pvi5WUWhsf4w7FJvt2herunrAVSP821FuTiCRncVK-jw0ey5JdEP7DShfFt_usVJvUUMNEFrG_m1EuDLCkn6V9LRBJYNPIWk2muJ2AYfKSKvgfGW_2nHC2DOzzQVSehcTBCHvNixWgMAQzlRhRO5qoJlrNZpb9fgwfFXWxkpeKkjLWZwkq7RZI0q65y-X90Et5UGI45x_yKBCtSZyl5pdlo4rY6A8HX9d_-6pCMD-yAkncK3Q8w5qlw7GMHz_bgKRSpC0rI8-ClsbyOOBNsbyOCPs2voCzRgtss6k0wyl8GebK_Ro_ORhNIt56ujnclaOv7keNKKoOcMor9GRNIezCZjXfINHc9O8EVs%3D&sjdu=QWF4TUFyrHvH7u082A4fa_aRX3M-UVWF5Ox5H4EyFBg&acatk=1g0hjl8p4t61o802&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=1940ced3abf7b4ef&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=1940ced3abf7b4ef&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=257c9b4c2b3594d5&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=257c9b4c2b3594d5&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=2bd8900b54705716&from=ja&exp=1&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=2bd8900b54705716&from=ja&exp=1&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=3992e838a646bef5&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=3992e838a646bef5&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=3e4629d9c1c86891&from=ja&exp=1&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=3e4629d9c1c86891&from=ja&exp=1&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=4a22e037d442607e&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=4a22e037d442607e&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=4abd867d9e000fc9&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7536404725441878&adid=387070476&ad=-6NYlbfkN0Dr_hnQ-Jfvx8x1kl6LZGw5tBy_C31iSfGQgc5_4VANfmnlk7wUsb9qjlaUE66B-57Xn_REJ0k3MUmohgE1H0I85KNgzlGFi8nVN6B3hjvJ29jLVlKKX2Kyd_2BgvxjR5konbnMsHyQ5dKZfkR4GnX6Gsnv7sIAj4CBl6tXh6n_BFvlj3nt7rFxpFCrBUVoXuTLHn1Gctgc-4oNtPWlsF0zZG_CHgq_DovfSIQnIcltEfpdPwutIj9wPM2H07zlp_jfZl1k4nhRg7aGJzWeFKm1jPVXc8IuvunxexR3f1ix9ljZT3W-Irr1sjifPjoJs57Mog2lIzVI-NPQxK9tMm8-V5fk8fJTN3HXTYzEt7aVzPrWqzFShwUlanbNBnXw2jQ%3D&sjdu=QWF4TUFyrHvH7u082A4fa88oBOkWeOCxRWGSuT_8J9I&acatk=1g0hpa59tmfpl800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=4abd867d9e000fc9&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7536404725441878&adid=387070476&ad=-6NYlbfkN0Dr_hnQ-Jfvx8x1kl6LZGw5tBy_C31iSfGQgc5_4VANfmnlk7wUsb9qjlaUE66B-57Xn_REJ0k3MUmohgE1H0I85KNgzlGFi8nVN6B3hjvJ29jLVlKKX2Kyd_2BgvxjR5konbnMsHyQ5dKZfkR4GnX6Gsnv7sIAj4CBl6tXh6n_BFvlj3nt7rFxpFCrBUVoXuTLHn1Gctgc-4oNtPWlsF0zZG_CHgq_DovfSIQnIcltEfpdPwutIj9wPM2H07zlp_jfZl1k4nhRg7aGJzWeFKm1jPVXc8IuvunxexR3f1ix9ljZT3W-Irr1sjifPjoJs57Mog2lIzVI-NPQxK9tMm8-V5fk8fJTN3HXTYzEt7aVzPrWqzFShwUlanbNBnXw2jQ%3D&sjdu=QWF4TUFyrHvH7u082A4fa88oBOkWeOCxRWGSuT_8J9I&acatk=1g0hpa59tmfpl800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=4b50dd57a68879ae&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7536404725441878&adid=387070476&ad=-6NYlbfkN0Dr_hnQ-Jfvx8x1kl6LZGw5tBy_C31iSfGQgc5_4VANfmnlk7wUsb9qjlaUE66B-57Xn_REJ0k3MYqfS1fhmTACCYtenzbRdGGtHp4lX0b66o4i3aEsgWByAQjMK8ev9a5VwTdV89Xlc3aae-xr1JUOYX_gckO-ta7ruPEYn_ETZIFEMQ4rgeGfy67JDYrGtXtM-Wnv7WD9gIk6N-WUOUUeQN_8Gk2nOnJv2U3OmapA7W_B3_qU-44ZM6NGOdFrVhBWcLpeVbt_BAxL5r61nbgxHA8pJ-T0xM37blcIaifV_-zPcMUStgaIBFy5ry4z5Z9bUYSRkww3LgD8UKzulfMJOHKEbRckSHZocu05GZMO3cxc3HlBCRtFBKja7PMLck4%3D&sjdu=QWF4TUFyrHvH7u082A4fa0bCtOJu3VoYDgQVv6KhZTs&acatk=1g0hp9g9bq6dq800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=4b50dd57a68879ae&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7536404725441878&adid=387070476&ad=-6NYlbfkN0Dr_hnQ-Jfvx8x1kl6LZGw5tBy_C31iSfGQgc5_4VANfmnlk7wUsb9qjlaUE66B-57Xn_REJ0k3MYqfS1fhmTACCYtenzbRdGGtHp4lX0b66o4i3aEsgWByAQjMK8ev9a5VwTdV89Xlc3aae-xr1JUOYX_gckO-ta7ruPEYn_ETZIFEMQ4rgeGfy67JDYrGtXtM-Wnv7WD9gIk6N-WUOUUeQN_8Gk2nOnJv2U3OmapA7W_B3_qU-44ZM6NGOdFrVhBWcLpeVbt_BAxL5r61nbgxHA8pJ-T0xM37blcIaifV_-zPcMUStgaIBFy5ry4z5Z9bUYSRkww3LgD8UKzulfMJOHKEbRckSHZocu05GZMO3cxc3HlBCRtFBKja7PMLck4%3D&sjdu=QWF4TUFyrHvH7u082A4fa0bCtOJu3VoYDgQVv6KhZTs&acatk=1g0hp9g9bq6dq800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=4b50dd57a68879ae&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7536404725441878&adid=387070476&ad=-6NYlbfkN0Dr_hnQ-Jfvx8x1kl6LZGw5tBy_C31iSfGQgc5_4VANfmnlk7wUsb9qjlaUE66B-57Xn_REJ0k3MYqfS1fhmTACCYtenzbRdGGtHp4lX0b66o4i3aEsgWByAQjMK8ev9a5VwTdV89Xlc3aae-xr1JUOYX_gckO-ta7ruPEYn_ETZIFEMQ4rgeGfy67JDYrGtXtM-Wnv7WD9gIk6N-WUOUUeQN_8Gk2nOnJv2U3OmapA7W_B3_qU-44ZM6NGOdFrVhBWcLpeVbt_BAxL5r61nbgxHA8pJ-T0xM37blcIaifV_-zPcMUStgaImEA6mOxMq5lDj7Rf5b0ND5QW9RgRXmHrUIvlAhvbGuW5nseuzmJOarnJmzOIjWAjpt4RNT9VIyc%3D&sjdu=QWF4TUFyrHvH7u082A4fa0bCtOJu3VoYDgQVv6KhZTs&acatk=1g0hkhp13ocqo800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=4b50dd57a68879ae&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7536404725441878&adid=387070476&ad=-6NYlbfkN0Dr_hnQ-Jfvx8x1kl6LZGw5tBy_C31iSfGQgc5_4VANfmnlk7wUsb9qjlaUE66B-57Xn_REJ0k3MYqfS1fhmTACCYtenzbRdGGtHp4lX0b66o4i3aEsgWByAQjMK8ev9a5VwTdV89Xlc3aae-xr1JUOYX_gckO-ta7ruPEYn_ETZIFEMQ4rgeGfy67JDYrGtXtM-Wnv7WD9gIk6N-WUOUUeQN_8Gk2nOnJv2U3OmapA7W_B3_qU-44ZM6NGOdFrVhBWcLpeVbt_BAxL5r61nbgxHA8pJ-T0xM37blcIaifV_-zPcMUStgaImEA6mOxMq5lDj7Rf5b0ND5QW9RgRXmHrUIvlAhvbGuW5nseuzmJOarnJmzOIjWAjpt4RNT9VIyc%3D&sjdu=QWF4TUFyrHvH7u082A4fa0bCtOJu3VoYDgQVv6KhZTs&acatk=1g0hkhp13ocqo800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=5980a4297fe83d02&q=data+scientist+$100,000&l=Tucson&tk=1fv19bniop1h1801&from=ja&advn=7695628155876325&adid=384499560&ad=-6NYlbfkN0AauYDK0PcpkAAwvqsYr42ytNXSoRmB0ySYhRIkJ-ozknMmzV10mP9D-ZXILu2789a1dSmA0S2zinv8eXSHFy3xh06QFu0iu2OyFskdyYg8L3Y_IJIrHvOmazO0bGUZFl6Y1XvKQ1cOzR4_b-pPtBYfXvCV40K8lwgU7LVAZoXMKpfIYWyjZYLSfLuQzJ9fyWYOfcQm9Dhd2HKy7axNAfWqx18CPFLJGKVicUjJE0H8oos6w0hgz6bctHHY_kOp_tZYup-594j12CAx_sZOWQcsq0QWou0z0SDZzsdEZsOMGwh1PMMrMSWCmiwd5f7n6n3ejBgQpZ41fUJCGnvE5c-3OysGvprb9NHjGWNvjWlCR8wqXJJClWve&sjdu=9nrDNPdV1DghkDNnC2WJlW1nrk21-asFvjSk9jx-s_PQq5REO_FJF1xrc-__R1RpjpUTGPWo1X4sMmWP-o_mjsecBCZ6U-3ANC9kJcMsr8AiBwSHYxA2hEX67nA2SsOLqNNE7CYG-1bRDUhtcfur3u8DM5i_gh9ufnYzHvtCpDLj6Vt8Qnj_2LJoleLDXPPr8U1xxKiKp1tLHtlRL9jPlqwoWk-VXHA9jjet2nyPNfRL-sAoI7Spr3TDuNgMT4lupULPUV3WW-kPb1wKjuVsr0V4r5ExK-dkbBi02YGc_zbHnAQmelPtwDQvZCXDLK_ANce0r3Dehaq3YeeATYvql_TU0D0bfrcQQUcZ3mYzZRnVrKb-GVB781bShguZFfVfdbVwxIngvGwIBb_kmTPdUo3fx8kbSEhpcgX71FtjyR44FuyebxQcN3HlMQqoG6D2nChZeBZJv6eKBSOPcP3-MWO7H8KQy77MnLnJ2LDa-3J8nBHPizaR0najtwZetXLkUb3_RpRexL5vqrFQDWfr3A&acatk=1fv1cmlve3oah000&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=5980a4297fe83d02&q=data+scientist+$100,000&l=Tucson&tk=1fv3p1b6vp1h1801&from=ja&advn=7695628155876325&adid=384499560&ad=-6NYlbfkN0AauYDK0PcpkAAwvqsYr42ytNXSoRmB0ySYhRIkJ-ozknMmzV10mP9D-ZXILu2789a1dSmA0S2zinv8eXSHFy3xh06QFu0iu2OyFskdyYg8L3Y_IJIrHvOmazO0bGUZFl4l7rGDiH3f1EWAZw35AYmnkSe2HwcxN4MuLJuSqWmH9pxNGPWCtn5yOaPZFmw-jKdWU0H1PLyVBR02GSJ_Y-nwOQx63PTvmf3pXdezhV6_aDQb5_l5578OxKLJk-MUQ-ZxFrNWT0dVCkycc25MYIljw27MC95MMATNDAy28RhA3yvGttMjEl9ZArBjsicjaFfZzugdyqyDqw_mDW-UrLS0CTar2WI7cAyiabRd-2YJ3g%3D%3D&sjdu=9nrDNPdV1DghkDNnC2WJlW1nrk21-asFvjSk9jx-s_PQq5REO_FJF1xrc-__R1RpjpUTGPWo1X4sMmWP-o_mjsecBCZ6U-3ANC9kJcMsr8AiBwSHYxA2hEX67nA2SsOLqNNE7CYG-1bRDUhtcfur3u8DM5i_gh9ufnYzHvtCpDLj6Vt8Qnj_2LJoleLDXPPr8U1xxKiKp1tLHtlRL9jPlqwoWk-VXHA9jjet2nyPNfRL-sAoI7Spr3TDuNgMT4lupULPUV3WW-kPb1wKjuVsr0V4r5ExK-dkbBi02YGc_zbHnAQmelPtwDQvZCXDLK_ANce0r3Dehaq3YeeATYvql_TU0D0bfrcQQUcZ3mYzZRnVrKb-GVB781bShguZFfVfP6fDDHCjDcuLJhzO6At9E43fx8kbSEhpcgX71FtjyR44FuyebxQcN3HlMQqoG6D2nChZeBZJv6eKBSOPcP3-MWO7H8KQy77MnLnJ2LDa-3J8nBHPizaR0najtwZetXLkUb3_RpRexL5vqrFQDWfr3A&acatk=1fv3ttv7q3h80002&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=5980a4297fe83d02&q=data+scientist+$100,000&l=Tucson&tk=1fv91q3m7pijt801&from=ja&advn=7695628155876325&adid=384499560&ad=-6NYlbfkN0AauYDK0PcpkAAwvqsYr42ytNXSoRmB0ySYhRIkJ-ozknMmzV10mP9D-ZXILu2789a1dSmA0S2zinv8eXSHFy3xh06QFu0iu2OyFskdyYg8L3Y_IJIrHvOmazO0bGUZFl6EmoK1MOaAwp_B-EV_3P0R4nss1uTpTyANhZ9xa2kJM1uPA793G3IJwq1V7JXa1Ylz0dKHNxInHOJD7NcLvG3qOiWk1Mho3j_WtaLxd8cQlmq5s0m-Pgk2ZZ5rhtXK8AIxRO6ihRsXMzY4q2_G9lU0dxmz4NTweWwsLrayaVq7oiSxhaMcVh_ImhuLYHH5m-eAWWSQC7RzUtXTvZuyEox9RPu00r3KGq9_SqLB5oJnXVoVkxIJy2dTFyX_Cd_PtgM%3D&sjdu=9nrDNPdV1DghkDNnC2WJlW1nrk21-asFvjSk9jx-s_PQq5REO_FJF1xrc-__R1RpjpUTGPWo1X4sMmWP-o_mjsecBCZ6U-3ANC9kJcMsr8AiBwSHYxA2hEX67nA2SsOLqNNE7CYG-1bRDUhtcfur3u8DM5i_gh9ufnYzHvtCpDLj6Vt8Qnj_2LJoleLDXPPr8U1xxKiKp1tLHtlRL9jPlqwoWk-VXHA9jjet2nyPNfRL-sAoI7Spr3TDuNgMT4lupULPUV3WW-kPb1wKjuVsr0V4r5ExK-dkbBi02YGc_zbHnAQmelPtwDQvZCXDLK_ANce0r3Dehaq3YeeATYvql_TU0D0bfrcQQUcZ3mYzZRnVrKb-GVB781bShguZFfVfbSg7lJn7niyS01VrJBvWUY3fx8kbSEhpcgX71FtjyR44FuyebxQcN3HlMQqoG6D2nChZeBZJv6eKBSOPcP3-MWO7H8KQy77MnLnJ2LDa-3J8nBHPizaR0najtwZetXLkUb3_RpRexL5vqrFQDWfr3A&acatk=1fv91vbhcu21s800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=5980a4297fe83d02&q=data+scientist+%24100%2C000&l=Tucson&tk=1fv19bniop1h1801&from=ja&advn=7695628155876325&adid=384499560&ad=-6NYlbfkN0AauYDK0PcpkAAwvqsYr42ytNXSoRmB0ySYhRIkJ-ozknMmzV10mP9D-ZXILu2789a1dSmA0S2zinv8eXSHFy3xh06QFu0iu2OyFskdyYg8L3Y_IJIrHvOmazO0bGUZFl6Y1XvKQ1cOzR4_b-pPtBYfXvCV40K8lwgU7LVAZoXMKpfIYWyjZYLSfLuQzJ9fyWYOfcQm9Dhd2HKy7axNAfWqx18CPFLJGKVicUjJE0H8oos6w0hgz6bctHHY_kOp_tZYup-594j12CAx_sZOWQcsq0QWou0z0SDZzsdEZsOMGwh1PMMrMSWCmiwd5f7n6n3ejBgQpZ41fUJCGnvE5c-3OysGvprb9NHjGWNvjWlCR8wqXJJClWve&sjdu=9nrDNPdV1DghkDNnC2WJlW1nrk21-asFvjSk9jx-s_PQq5REO_FJF1xrc-__R1RpjpUTGPWo1X4sMmWP-o_mjsecBCZ6U-3ANC9kJcMsr8AiBwSHYxA2hEX67nA2SsOLqNNE7CYG-1bRDUhtcfur3u8DM5i_gh9ufnYzHvtCpDLj6Vt8Qnj_2LJoleLDXPPr8U1xxKiKp1tLHtlRL9jPlqwoWk-VXHA9jjet2nyPNfRL-sAoI7Spr3TDuNgMT4lupULPUV3WW-kPb1wKjuVsr0V4r5ExK-dkbBi02YGc_zbHnAQmelPtwDQvZCXDLK_ANce0r3Dehaq3YeeATYvql_TU0D0bfrcQQUcZ3mYzZRnVrKb-GVB781bShguZFfVfdbVwxIngvGwIBb_kmTPdUo3fx8kbSEhpcgX71FtjyR44FuyebxQcN3HlMQqoG6D2nChZeBZJv6eKBSOPcP3-MWO7H8KQy77MnLnJ2LDa-3J8nBHPizaR0najtwZetXLkUb3_RpRexL5vqrFQDWfr3A&acatk=1fv1cmlve3oah000&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=5980a4297fe83d02&q=data+scientist+%24100%2C000&l=Tucson&tk=1fv19bniop1h1801&from=ja&advn=7695628155876325&adid=384499560&ad=-6NYlbfkN0AauYDK0PcpkAAwvqsYr42ytNXSoRmB0ySYhRIkJ-ozknMmzV10mP9D-ZXILu2789a1dSmA0S2zinv8eXSHFy3xh06QFu0iu2OyFskdyYg8L3Y_IJIrHvOmazO0bGUZFl6Y1XvKQ1cOzR4_b-pPtBYfXvCV40K8lwgU7LVAZoXMKpfIYWyjZYLSfLuQzJ9fyWYOfcQm9Dhd2HKy7axNAfWqx18CPFLJGKVicUjJE0H8oos6w0hgz6bctHHY_kOp_tZYup-594j12CAx_sZOWQcsq0QWou0z0SDZzsdEZsOMGwh1PMMrMSWCmiwd5f7n6n3ejBgQpZ41fUJCGnvE5c-3OysGvprb9NHjGWNvjWlCR8wqXJJClWve&sjdu=9nrDNPdV1DghkDNnC2WJlW1nrk21-asFvjSk9jx-s_PQq5REO_FJF1xrc-__R1RpjpUTGPWo1X4sMmWP-o_mjsecBCZ6U-3ANC9kJcMsr8AiBwSHYxA2hEX67nA2SsOLqNNE7CYG-1bRDUhtcfur3u8DM5i_gh9ufnYzHvtCpDLj6Vt8Qnj_2LJoleLDXPPr8U1xxKiKp1tLHtlRL9jPlqwoWk-VXHA9jjet2nyPNfRL-sAoI7Spr3TDuNgMT4lupULPUV3WW-kPb1wKjuVsr0V4r5ExK-dkbBi02YGc_zbHnAQmelPtwDQvZCXDLK_ANce0r3Dehaq3YeeATYvql_TU0D0bfrcQQUcZ3mYzZRnVrKb-GVB781bShguZFfVfdbVwxIngvGwIBb_kmTPdUo3fx8kbSEhpcgX71FtjyR44FuyebxQcN3HlMQqoG6D2nChZeBZJv6eKBSOPcP3-MWO7H8KQy77MnLnJ2LDa-3J8nBHPizaR0najtwZetXLkUb3_RpRexL5vqrFQDWfr3A&acatk=1fv1cmlve3oah000&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=5980a4297fe83d02&q=data+scientist+%24100%2C000&l=Tucson&tk=1fv3p1b6vp1h1801&from=ja&advn=7695628155876325&adid=384499560&ad=-6NYlbfkN0AauYDK0PcpkAAwvqsYr42ytNXSoRmB0ySYhRIkJ-ozknMmzV10mP9D-ZXILu2789a1dSmA0S2zinv8eXSHFy3xh06QFu0iu2OyFskdyYg8L3Y_IJIrHvOmazO0bGUZFl4l7rGDiH3f1EWAZw35AYmnkSe2HwcxN4MuLJuSqWmH9pxNGPWCtn5yOaPZFmw-jKdWU0H1PLyVBR02GSJ_Y-nwOQx63PTvmf3pXdezhV6_aDQb5_l5578OxKLJk-MUQ-ZxFrNWT0dVCkycc25MYIljw27MC95MMATNDAy28RhA3yvGttMjEl9ZArBjsicjaFfZzugdyqyDqw_mDW-UrLS0CTar2WI7cAyiabRd-2YJ3g%3D%3D&sjdu=9nrDNPdV1DghkDNnC2WJlW1nrk21-asFvjSk9jx-s_PQq5REO_FJF1xrc-__R1RpjpUTGPWo1X4sMmWP-o_mjsecBCZ6U-3ANC9kJcMsr8AiBwSHYxA2hEX67nA2SsOLqNNE7CYG-1bRDUhtcfur3u8DM5i_gh9ufnYzHvtCpDLj6Vt8Qnj_2LJoleLDXPPr8U1xxKiKp1tLHtlRL9jPlqwoWk-VXHA9jjet2nyPNfRL-sAoI7Spr3TDuNgMT4lupULPUV3WW-kPb1wKjuVsr0V4r5ExK-dkbBi02YGc_zbHnAQmelPtwDQvZCXDLK_ANce0r3Dehaq3YeeATYvql_TU0D0bfrcQQUcZ3mYzZRnVrKb-GVB781bShguZFfVfP6fDDHCjDcuLJhzO6At9E43fx8kbSEhpcgX71FtjyR44FuyebxQcN3HlMQqoG6D2nChZeBZJv6eKBSOPcP3-MWO7H8KQy77MnLnJ2LDa-3J8nBHPizaR0najtwZetXLkUb3_RpRexL5vqrFQDWfr3A&acatk=1fv3ttv7q3h80002&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=5980a4297fe83d02&q=data+scientist+%24100%2C000&l=Tucson&tk=1fv3p1b6vp1h1801&from=ja&advn=7695628155876325&adid=384499560&ad=-6NYlbfkN0AauYDK0PcpkAAwvqsYr42ytNXSoRmB0ySYhRIkJ-ozknMmzV10mP9D-ZXILu2789a1dSmA0S2zinv8eXSHFy3xh06QFu0iu2OyFskdyYg8L3Y_IJIrHvOmazO0bGUZFl4l7rGDiH3f1EWAZw35AYmnkSe2HwcxN4MuLJuSqWmH9pxNGPWCtn5yOaPZFmw-jKdWU0H1PLyVBR02GSJ_Y-nwOQx63PTvmf3pXdezhV6_aDQb5_l5578OxKLJk-MUQ-ZxFrNWT0dVCkycc25MYIljw27MC95MMATNDAy28RhA3yvGttMjEl9ZArBjsicjaFfZzugdyqyDqw_mDW-UrLS0CTar2WI7cAyiabRd-2YJ3g%3D%3D&sjdu=9nrDNPdV1DghkDNnC2WJlW1nrk21-asFvjSk9jx-s_PQq5REO_FJF1xrc-__R1RpjpUTGPWo1X4sMmWP-o_mjsecBCZ6U-3ANC9kJcMsr8AiBwSHYxA2hEX67nA2SsOLqNNE7CYG-1bRDUhtcfur3u8DM5i_gh9ufnYzHvtCpDLj6Vt8Qnj_2LJoleLDXPPr8U1xxKiKp1tLHtlRL9jPlqwoWk-VXHA9jjet2nyPNfRL-sAoI7Spr3TDuNgMT4lupULPUV3WW-kPb1wKjuVsr0V4r5ExK-dkbBi02YGc_zbHnAQmelPtwDQvZCXDLK_ANce0r3Dehaq3YeeATYvql_TU0D0bfrcQQUcZ3mYzZRnVrKb-GVB781bShguZFfVfP6fDDHCjDcuLJhzO6At9E43fx8kbSEhpcgX71FtjyR44FuyebxQcN3HlMQqoG6D2nChZeBZJv6eKBSOPcP3-MWO7H8KQy77MnLnJ2LDa-3J8nBHPizaR0najtwZetXLkUb3_RpRexL5vqrFQDWfr3A&acatk=1fv3ttv7q3h80002&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=5980a4297fe83d02&q=data+scientist+%24100%2C000&l=Tucson&tk=1fv91q3m7pijt801&from=ja&advn=7695628155876325&adid=384499560&ad=-6NYlbfkN0AauYDK0PcpkAAwvqsYr42ytNXSoRmB0ySYhRIkJ-ozknMmzV10mP9D-ZXILu2789a1dSmA0S2zinv8eXSHFy3xh06QFu0iu2OyFskdyYg8L3Y_IJIrHvOmazO0bGUZFl6EmoK1MOaAwp_B-EV_3P0R4nss1uTpTyANhZ9xa2kJM1uPA793G3IJwq1V7JXa1Ylz0dKHNxInHOJD7NcLvG3qOiWk1Mho3j_WtaLxd8cQlmq5s0m-Pgk2ZZ5rhtXK8AIxRO6ihRsXMzY4q2_G9lU0dxmz4NTweWwsLrayaVq7oiSxhaMcVh_ImhuLYHH5m-eAWWSQC7RzUtXTvZuyEox9RPu00r3KGq9_SqLB5oJnXVoVkxIJy2dTFyX_Cd_PtgM%3D&sjdu=9nrDNPdV1DghkDNnC2WJlW1nrk21-asFvjSk9jx-s_PQq5REO_FJF1xrc-__R1RpjpUTGPWo1X4sMmWP-o_mjsecBCZ6U-3ANC9kJcMsr8AiBwSHYxA2hEX67nA2SsOLqNNE7CYG-1bRDUhtcfur3u8DM5i_gh9ufnYzHvtCpDLj6Vt8Qnj_2LJoleLDXPPr8U1xxKiKp1tLHtlRL9jPlqwoWk-VXHA9jjet2nyPNfRL-sAoI7Spr3TDuNgMT4lupULPUV3WW-kPb1wKjuVsr0V4r5ExK-dkbBi02YGc_zbHnAQmelPtwDQvZCXDLK_ANce0r3Dehaq3YeeATYvql_TU0D0bfrcQQUcZ3mYzZRnVrKb-GVB781bShguZFfVfbSg7lJn7niyS01VrJBvWUY3fx8kbSEhpcgX71FtjyR44FuyebxQcN3HlMQqoG6D2nChZeBZJv6eKBSOPcP3-MWO7H8KQy77MnLnJ2LDa-3J8nBHPizaR0najtwZetXLkUb3_RpRexL5vqrFQDWfr3A&acatk=1fv91vbhcu21s800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=5980a4297fe83d02&q=data+scientist+%24100%2C000&l=Tucson&tk=1fv91q3m7pijt801&from=ja&advn=7695628155876325&adid=384499560&ad=-6NYlbfkN0AauYDK0PcpkAAwvqsYr42ytNXSoRmB0ySYhRIkJ-ozknMmzV10mP9D-ZXILu2789a1dSmA0S2zinv8eXSHFy3xh06QFu0iu2OyFskdyYg8L3Y_IJIrHvOmazO0bGUZFl6EmoK1MOaAwp_B-EV_3P0R4nss1uTpTyANhZ9xa2kJM1uPA793G3IJwq1V7JXa1Ylz0dKHNxInHOJD7NcLvG3qOiWk1Mho3j_WtaLxd8cQlmq5s0m-Pgk2ZZ5rhtXK8AIxRO6ihRsXMzY4q2_G9lU0dxmz4NTweWwsLrayaVq7oiSxhaMcVh_ImhuLYHH5m-eAWWSQC7RzUtXTvZuyEox9RPu00r3KGq9_SqLB5oJnXVoVkxIJy2dTFyX_Cd_PtgM%3D&sjdu=9nrDNPdV1DghkDNnC2WJlW1nrk21-asFvjSk9jx-s_PQq5REO_FJF1xrc-__R1RpjpUTGPWo1X4sMmWP-o_mjsecBCZ6U-3ANC9kJcMsr8AiBwSHYxA2hEX67nA2SsOLqNNE7CYG-1bRDUhtcfur3u8DM5i_gh9ufnYzHvtCpDLj6Vt8Qnj_2LJoleLDXPPr8U1xxKiKp1tLHtlRL9jPlqwoWk-VXHA9jjet2nyPNfRL-sAoI7Spr3TDuNgMT4lupULPUV3WW-kPb1wKjuVsr0V4r5ExK-dkbBi02YGc_zbHnAQmelPtwDQvZCXDLK_ANce0r3Dehaq3YeeATYvql_TU0D0bfrcQQUcZ3mYzZRnVrKb-GVB781bShguZFfVfbSg7lJn7niyS01VrJBvWUY3fx8kbSEhpcgX71FtjyR44FuyebxQcN3HlMQqoG6D2nChZeBZJv6eKBSOPcP3-MWO7H8KQy77MnLnJ2LDa-3J8nBHPizaR0najtwZetXLkUb3_RpRexL5vqrFQDWfr3A&acatk=1fv91vbhcu21s800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=5c472141416e370b&q=predictive+modeler&l=Remote&tk=1g05hrav6stu4800&from=ja&alid=624c438eb87569687208359f&rgtk=1g05hrav6stu4800',
    'https://www.indeed.com/viewjob?jk=5c472141416e370b&q=predictive+modeler&l=Remote&tk=1g05hrav6stu4800&from=ja&alid=624c438eb87569687208359f&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g05hrav6stu4800',
    'https://www.indeed.com/viewjob?jk=71186e1061253171&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=71186e1061253171&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=810788582b2287ca&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7536404725441878&adid=387070476&ad=-6NYlbfkN0Dr_hnQ-Jfvx8x1kl6LZGw5tBy_C31iSfGQgc5_4VANfmnlk7wUsb9qjlaUE66B-57Xn_REJ0k3MTL9KizYiFQQbigYQvLaIuABwhVExVrStZsoKF6tBevnG7PEk71LkNJnVrEmHoNIshJd1I26Yf7f_rNrs1LZdnMGRylQK2pNAeS02X5bxXwOx7e8cfaEQK-0c29_ThnKd78ZcrwJmx137AZdZOZRJ28WVB2CAJrJSgXnqygWtLk-h8yLmxZ_ukJAdJUqYSUH4tpR95-AbCSzFteEoCvcaMU1MyrWTgYlOi-wCHDXmhM5whGgUh9P4rMomQfGot5A2KknVCvohwCfKbu_4foY7gdyL6T5mzACixbkKZVfd4gQkK9fJwfQgHc%3D&sjdu=QWF4TUFyrHvH7u082A4fa3a59T-jp1_jJTl6cj9RwxA&acatk=1g0hpa26jqt61800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=810788582b2287ca&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7536404725441878&adid=387070476&ad=-6NYlbfkN0Dr_hnQ-Jfvx8x1kl6LZGw5tBy_C31iSfGQgc5_4VANfmnlk7wUsb9qjlaUE66B-57Xn_REJ0k3MTL9KizYiFQQbigYQvLaIuABwhVExVrStZsoKF6tBevnG7PEk71LkNJnVrEmHoNIshJd1I26Yf7f_rNrs1LZdnMGRylQK2pNAeS02X5bxXwOx7e8cfaEQK-0c29_ThnKd78ZcrwJmx137AZdZOZRJ28WVB2CAJrJSgXnqygWtLk-h8yLmxZ_ukJAdJUqYSUH4tpR95-AbCSzFteEoCvcaMU1MyrWTgYlOi-wCHDXmhM5whGgUh9P4rMomQfGot5A2KknVCvohwCfKbu_4foY7gdyL6T5mzACixbkKZVfd4gQkK9fJwfQgHc%3D&sjdu=QWF4TUFyrHvH7u082A4fa3a59T-jp1_jJTl6cj9RwxA&acatk=1g0hpa26jqt61800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=854d42fd8d9b0afc&q=data+scientist+$100,000&l=Tucson&tk=1fv19bniop1h1801&from=ja&advn=9078100093439348&adid=378099455&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUed2Tb_D9DpZa2YDqoHp_oOFTKST0XDKgYv5Qy1D9QijxJ4xVuLwl_3n9EW57UTZ1n_0Ki4g_7a2-1xnHW0DlOekRSjUK6TMM8421tRaHNKCEj2f_HMxEz8r4p_jykVt1wmGWm7iVkgE0iAe8vjzwyCgSv7GCwt1VtqZem6Z5uVrYtVQ9dOroBd5ha_wPrGva-vVdDU1LDoZVont4jHY1ui6vYwEbU6uqlGz0Qrkr7bWfjX8l4vsdlSSc7ZMKhTtTkss8hWMSTqG2fccLmOnuZIdVupfKbYe3D802IJRefGu60hceiViJyhI5f0qtWos96E%3D&sjdu=LClFkqgtkW6SKo_RfYnE68MkziOJDMuPgd4jONrp9lg_JyRSR6QuLWzF9h_xIauSg7IWDF0nJTMQ9h_mUx0iXLjWVtXXVUup2wlfa9oMVEN_inxkrx_tEhg4aFijCTtTIZC07ygPhF91ZX47Vbtk_r1rWC3swTmD46cR8JBQQeKgWpZRFfMj5VGDNYahLdIkmegMRtcm8uuOY2IEDud7A6S8uK_wrht1n5qcA_VPcPgMLl-5PxGs1lf8GzpTZ8kixUyfko6hHqWyg2TvvpNDRY62AMgvXgChptoEL2Itjrw&acatk=1fv1conug3h80000&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=854d42fd8d9b0afc&q=data+scientist+%24100%2C000&l=Tucson&tk=1fv19bniop1h1801&from=ja&advn=9078100093439348&adid=378099455&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUed2Tb_D9DpZa2YDqoHp_oOFTKST0XDKgYv5Qy1D9QijxJ4xVuLwl_3n9EW57UTZ1n_0Ki4g_7a2-1xnHW0DlOekRSjUK6TMM8421tRaHNKCEj2f_HMxEz8r4p_jykVt1wmGWm7iVkgE0iAe8vjzwyCgSv7GCwt1VtqZem6Z5uVrYtVQ9dOroBd5ha_wPrGva-vVdDU1LDoZVont4jHY1ui6vYwEbU6uqlGz0Qrkr7bWfjX8l4vsdlSSc7ZMKhTtTkss8hWMSTqG2fccLmOnuZIdVupfKbYe3D802IJRefGu60hceiViJyhI5f0qtWos96E%3D&sjdu=LClFkqgtkW6SKo_RfYnE68MkziOJDMuPgd4jONrp9lg_JyRSR6QuLWzF9h_xIauSg7IWDF0nJTMQ9h_mUx0iXLjWVtXXVUup2wlfa9oMVEN_inxkrx_tEhg4aFijCTtTIZC07ygPhF91ZX47Vbtk_r1rWC3swTmD46cR8JBQQeKgWpZRFfMj5VGDNYahLdIkmegMRtcm8uuOY2IEDud7A6S8uK_wrht1n5qcA_VPcPgMLl-5PxGs1lf8GzpTZ8kixUyfko6hHqWyg2TvvpNDRY62AMgvXgChptoEL2Itjrw&acatk=1fv1conug3h80000&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=854d42fd8d9b0afc&q=data+scientist+%24100%2C000&l=Tucson&tk=1fv19bniop1h1801&from=ja&advn=9078100093439348&adid=378099455&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUed2Tb_D9DpZa2YDqoHp_oOFTKST0XDKgYv5Qy1D9QijxJ4xVuLwl_3n9EW57UTZ1n_0Ki4g_7a2-1xnHW0DlOekRSjUK6TMM8421tRaHNKCEj2f_HMxEz8r4p_jykVt1wmGWm7iVkgE0iAe8vjzwyCgSv7GCwt1VtqZem6Z5uVrYtVQ9dOroBd5ha_wPrGva-vVdDU1LDoZVont4jHY1ui6vYwEbU6uqlGz0Qrkr7bWfjX8l4vsdlSSc7ZMKhTtTkss8hWMSTqG2fccLmOnuZIdVupfKbYe3D802IJRefGu60hceiViJyhI5f0qtWos96E%3D&sjdu=LClFkqgtkW6SKo_RfYnE68MkziOJDMuPgd4jONrp9lg_JyRSR6QuLWzF9h_xIauSg7IWDF0nJTMQ9h_mUx0iXLjWVtXXVUup2wlfa9oMVEN_inxkrx_tEhg4aFijCTtTIZC07ygPhF91ZX47Vbtk_r1rWC3swTmD46cR8JBQQeKgWpZRFfMj5VGDNYahLdIkmegMRtcm8uuOY2IEDud7A6S8uK_wrht1n5qcA_VPcPgMLl-5PxGs1lf8GzpTZ8kixUyfko6hHqWyg2TvvpNDRY62AMgvXgChptoEL2Itjrw&acatk=1fv1conug3h80000&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=8fc64766994e1bf5&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=8fc64766994e1bf5&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=9265ce23af9f1cf6&q=data%20scientist%20%24100%2C000&l=Tucson&tk=1fvocupudhv6q803&from=ja%2CiaBackPress&advn=3515783651855859&adid=376643618&ad=-6NYlbfkN0D3QdlsyYb-CFd0gdLB_D-feLwOvyjf8jHr365ieMnNNcRyQ1gdiqJcgFT6RNx_-oKJ7Z5WJFJg7tJRrFoYwlBJJQnD7PSITEE-IQkWjy0t9DTFYgU7ZvN5btnvvCmpZL_RL6l6QkRYHBzKNSvMVXxZ5-pyUS0Q1MVArrysNWJIvidTFtSZPA3ZBOMrzMVlwhKhW_7YMs88eyCri9o0wGq5fvSFM7jF9pf1CBjaXe2Jg5XKhN0k2y-du2l329RqZ3b1ZkTFmfhu7dbE8R7WxmM0dMJZg6TQf-CxtQg7DEl-HSOSuoj1QHcBcZE33UUU-QokYaF4NvtkcTsOeVZ7bGOqBxens588cjemEbSX-bDH_QQlnoq9YfcxRf3JsO1aVtDys3o4FswYRg%3D%3D&sjdu=8aKTnhAUxh5ENd4kai56sqSM1f1aHYVaLBGbwzgdtjFTWaqsxIzov5VIPb6ACBB1aF2E0oro296wQtwqUByUcVk0s-D07AErnaN1RI_K3NAJFs7R_1aw1crSrPJwpQsX&acatk=1fvq2gp07s7m8800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=9265ce23af9f1cf6&q=data+scientist+$100,000&l=Tucson&tk=1fvjcie8bhv6q801&from=ja&advn=3515783651855859&adid=376643618&ad=-6NYlbfkN0D3QdlsyYb-CFd0gdLB_D-feLwOvyjf8jHr365ieMnNNcRyQ1gdiqJcgFT6RNx_-oKJ7Z5WJFJg7tJRrFoYwlBJJQnD7PSITEE-IQkWjy0t9DTFYgU7ZvN5btnvvCmpZL_RL6l6QkRYHAvFYMyRtSfnyqaZNAFZs_3d47rK49Ss51G4aq2VTZGrSc5Ox_Y59xcvBSQaql0GPGUJ4bAkChkt0haJSO_BOBh2l8M49IAnMMKgVSyUtz3HZFlCtghZnu9WH7JGaXAXr86AKjVrbdbEPISw4_Q9aPyv4Nol6-vKYd31MuGnqLvWHZozfmH7SGEM1VY1i3dj-_wSkZH59oHD9Hf0g8yZpxBnYOTp5YRyPcDMgJ5FswAM3Zfia_1pZnbSMNAb5WMgCw%3D%3D&sjdu=8aKTnhAUxh5ENd4kai56sqSM1f1aHYVaLBGbwzgdtjFTWaqsxIzov5VIPb6ACBB1aF2E0oro296wQtwqUByUcVk0s-D07AErnaN1RI_K3NAJFs7R_1aw1crSrPJwpQsX&acatk=1fvjepvt7t4gq800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=9265ce23af9f1cf6&q=data+scientist+$100,000&l=Tucson&tk=1fvocupudhv6q803&from=ja&advn=3515783651855859&adid=376643618&ad=-6NYlbfkN0D3QdlsyYb-CFd0gdLB_D-feLwOvyjf8jHr365ieMnNNcRyQ1gdiqJcgFT6RNx_-oKJ7Z5WJFJg7tJRrFoYwlBJJQnD7PSITEE-IQkWjy0t9DTFYgU7ZvN5btnvvCmpZL_RL6l6QkRYHBzKNSvMVXxZ5-pyUS0Q1MVArrysNWJIvidTFtSZPA3ZBOMrzMVlwhKhW_7YMs88eyCri9o0wGq5fvSFM7jF9pf1CBjaXe2Jg5XKhN0k2y-du2l329RqZ3b1ZkTFmfhu7dbE8R7WxmM0dMJZg6TQf-CxtQg7DEl-HSOSuoj1QHcBcZE33UUU-QokYaF4NvtkcTsOeVZ7bGOqBxens588cjemEbSX-bDH_QQlnoq9YfcxRf3JsO1aVtDys3o4FswYRg%3D%3D&sjdu=8aKTnhAUxh5ENd4kai56sqSM1f1aHYVaLBGbwzgdtjFTWaqsxIzov5VIPb6ACBB1aF2E0oro296wQtwqUByUcVk0s-D07AErnaN1RI_K3NAJFs7R_1aw1crSrPJwpQsX&acatk=1fvq2gp07s7m8800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=9265ce23af9f1cf6&q=data+scientist+%24100%2C000&l=Tucson&tk=1fvjcie8bhv6q801&from=ja&advn=3515783651855859&adid=376643618&ad=-6NYlbfkN0D3QdlsyYb-CFd0gdLB_D-feLwOvyjf8jHr365ieMnNNcRyQ1gdiqJcgFT6RNx_-oKJ7Z5WJFJg7tJRrFoYwlBJJQnD7PSITEE-IQkWjy0t9DTFYgU7ZvN5btnvvCmpZL_RL6l6QkRYHAvFYMyRtSfnyqaZNAFZs_3d47rK49Ss51G4aq2VTZGrSc5Ox_Y59xcvBSQaql0GPGUJ4bAkChkt0haJSO_BOBh2l8M49IAnMMKgVSyUtz3HZFlCtghZnu9WH7JGaXAXr86AKjVrbdbEPISw4_Q9aPyv4Nol6-vKYd31MuGnqLvWHZozfmH7SGEM1VY1i3dj-_wSkZH59oHD9Hf0g8yZpxBnYOTp5YRyPcDMgJ5FswAM3Zfia_1pZnbSMNAb5WMgCw%3D%3D&sjdu=8aKTnhAUxh5ENd4kai56sqSM1f1aHYVaLBGbwzgdtjFTWaqsxIzov5VIPb6ACBB1aF2E0oro296wQtwqUByUcVk0s-D07AErnaN1RI_K3NAJFs7R_1aw1crSrPJwpQsX&acatk=1fvjepvt7t4gq800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=9265ce23af9f1cf6&q=data+scientist+%24100%2C000&l=Tucson&tk=1fvjcie8bhv6q801&from=ja&advn=3515783651855859&adid=376643618&ad=-6NYlbfkN0D3QdlsyYb-CFd0gdLB_D-feLwOvyjf8jHr365ieMnNNcRyQ1gdiqJcgFT6RNx_-oKJ7Z5WJFJg7tJRrFoYwlBJJQnD7PSITEE-IQkWjy0t9DTFYgU7ZvN5btnvvCmpZL_RL6l6QkRYHAvFYMyRtSfnyqaZNAFZs_3d47rK49Ss51G4aq2VTZGrSc5Ox_Y59xcvBSQaql0GPGUJ4bAkChkt0haJSO_BOBh2l8M49IAnMMKgVSyUtz3HZFlCtghZnu9WH7JGaXAXr86AKjVrbdbEPISw4_Q9aPyv4Nol6-vKYd31MuGnqLvWHZozfmH7SGEM1VY1i3dj-_wSkZH59oHD9Hf0g8yZpxBnYOTp5YRyPcDMgJ5FswAM3Zfia_1pZnbSMNAb5WMgCw%3D%3D&sjdu=8aKTnhAUxh5ENd4kai56sqSM1f1aHYVaLBGbwzgdtjFTWaqsxIzov5VIPb6ACBB1aF2E0oro296wQtwqUByUcVk0s-D07AErnaN1RI_K3NAJFs7R_1aw1crSrPJwpQsX&acatk=1fvjepvt7t4gq800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=9265ce23af9f1cf6&q=data+scientist+%24100%2C000&l=Tucson&tk=1fvocupudhv6q803&from=ja&advn=3515783651855859&adid=376643618&ad=-6NYlbfkN0D3QdlsyYb-CFd0gdLB_D-feLwOvyjf8jHr365ieMnNNcRyQ1gdiqJcgFT6RNx_-oKJ7Z5WJFJg7tJRrFoYwlBJJQnD7PSITEE-IQkWjy0t9DTFYgU7ZvN5btnvvCmpZL_RL6l6QkRYHBzKNSvMVXxZ5-pyUS0Q1MVArrysNWJIvidTFtSZPA3ZBOMrzMVlwhKhW_7YMs88eyCri9o0wGq5fvSFM7jF9pf1CBjaXe2Jg5XKhN0k2y-du2l329RqZ3b1ZkTFmfhu7dbE8R7WxmM0dMJZg6TQf-CxtQg7DEl-HSOSuoj1QHcBcZE33UUU-QokYaF4NvtkcTsOeVZ7bGOqBxens588cjemEbSX-bDH_QQlnoq9YfcxRf3JsO1aVtDys3o4FswYRg%3D%3D&sjdu=8aKTnhAUxh5ENd4kai56sqSM1f1aHYVaLBGbwzgdtjFTWaqsxIzov5VIPb6ACBB1aF2E0oro296wQtwqUByUcVk0s-D07AErnaN1RI_K3NAJFs7R_1aw1crSrPJwpQsX&acatk=1fvq2gp07s7m8800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=9265ce23af9f1cf6&q=data+scientist+%24100%2C000&l=Tucson&tk=1fvocupudhv6q803&from=ja&advn=3515783651855859&adid=376643618&ad=-6NYlbfkN0D3QdlsyYb-CFd0gdLB_D-feLwOvyjf8jHr365ieMnNNcRyQ1gdiqJcgFT6RNx_-oKJ7Z5WJFJg7tJRrFoYwlBJJQnD7PSITEE-IQkWjy0t9DTFYgU7ZvN5btnvvCmpZL_RL6l6QkRYHBzKNSvMVXxZ5-pyUS0Q1MVArrysNWJIvidTFtSZPA3ZBOMrzMVlwhKhW_7YMs88eyCri9o0wGq5fvSFM7jF9pf1CBjaXe2Jg5XKhN0k2y-du2l329RqZ3b1ZkTFmfhu7dbE8R7WxmM0dMJZg6TQf-CxtQg7DEl-HSOSuoj1QHcBcZE33UUU-QokYaF4NvtkcTsOeVZ7bGOqBxens588cjemEbSX-bDH_QQlnoq9YfcxRf3JsO1aVtDys3o4FswYRg%3D%3D&sjdu=8aKTnhAUxh5ENd4kai56sqSM1f1aHYVaLBGbwzgdtjFTWaqsxIzov5VIPb6ACBB1aF2E0oro296wQtwqUByUcVk0s-D07AErnaN1RI_K3NAJFs7R_1aw1crSrPJwpQsX&acatk=1fvq2gp07s7m8800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=999a7175bfaeee0c&q=python+jupyter&l=Tucson&tk=1g0cr0qkahv6q803&from=ja&advn=5550347756246289&adid=387601889&ad=-6NYlbfkN0BeTVAYNccvDEmPph7gZYfEOxyr-xfXmbJSPMpMvIORMf9jythPQ5rpqGth5Fnvc66aC9vOKGHFexzwSwWo88zrWr0EIimSRCPw1YssmqbDt1Bed7uA_t65hcBeTl0j-z90b_ijDZmZt6VcoSmM1Q4Jy9ussgGD7QYBdAL0N_CDTXQU0ArUVArdkoOVq5g0Rltau3QD7uPgqjjspyWo5A26UUZaM7ktbFx_vyQAv_yqNckztTCp49Zdfsr2_D9BbToURPZgcu_VJtLTVGubfdruOBUVN4N2vbq_96WTQ8oT9GRdqPA4c87F8TUrJm1V4Ej6vSFUHHcb9uEQW34dpH1JbgmuJqmbrVfBs6d6oO1egvmiBWqobRiXR0DxeeJT7o24M91dR1tmSw%3D%3D&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=999a7175bfaeee0c&q=python+jupyter&l=Tucson&tk=1g0cr0qkahv6q803&from=ja&advn=5550347756246289&adid=387601889&ad=-6NYlbfkN0BeTVAYNccvDEmPph7gZYfEOxyr-xfXmbJSPMpMvIORMf9jythPQ5rpqGth5Fnvc66aC9vOKGHFexzwSwWo88zrWr0EIimSRCPw1YssmqbDt1Bed7uA_t65hcBeTl0j-z90b_ijDZmZt6VcoSmM1Q4Jy9ussgGD7QYBdAL0N_CDTXQU0ArUVArdkoOVq5g0Rltau3QD7uPgqjjspyWo5A26UUZaM7ktbFx_vyQAv_yqNckztTCp49Zdfsr2_D9BbToURPZgcu_VJtLTVGubfdruOBUVN4N2vbq_96WTQ8oT9GRdqPA4c87F8TUrJm1V4Ej6vSFUHHcb9uEQW34dpH1JbgmuJqmbrVfBs6d6oO1egvmiBWqobRiXR0DxeeJT7o24M91dR1tmSw%3D%3D&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=999a7175bfaeee0c&q=python+jupyter&l=Tucson&tk=1g0cr0qkahv6q803&from=ja&advn=5550347756246289&adid=387601889&ad=-6NYlbfkN0BeTVAYNccvDEmPph7gZYfEOxyr-xfXmbJSPMpMvIORMf9jythPQ5rpqGth5Fnvc66aC9vOKGHFexzwSwWo88zrWr0EIimSRCPw1YssmqbDt1Bed7uA_t65hcBeTl0j-z90b_ijDZmZt6VcoSmM1Q4Jy9ussgGD7QYBdAL0N_CDTXQU0ArUVArdkoOVq5g0Rltau3QD7uPgqjjspyWo5A26UUZaM7ktbFx_vyQAv_yqNckztTCp49Zdfsr2_D9BbToURPZgcu_VJtLTVGubfdruOBUVN4N2vbq_96WTQ8oT9GRdqPA4c87F8TUrJm1V4Ejp-HfS-f2dcO9vWr08aQDDVUzzwUAL_cF1_YulAMydKQEPV8tvpm5PLfwtfauzHOlkZ_1hBQJf5A%3D%3D&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=999a7175bfaeee0c&q=python+jupyter&l=Tucson&tk=1g0cr0qkahv6q803&from=ja&advn=5550347756246289&adid=387601889&ad=-6NYlbfkN0BeTVAYNccvDEmPph7gZYfEOxyr-xfXmbJSPMpMvIORMf9jythPQ5rpqGth5Fnvc66aC9vOKGHFexzwSwWo88zrWr0EIimSRCPw1YssmqbDt1Bed7uA_t65hcBeTl0j-z90b_ijDZmZt6VcoSmM1Q4Jy9ussgGD7QYBdAL0N_CDTXQU0ArUVArdkoOVq5g0Rltau3QD7uPgqjjspyWo5A26UUZaM7ktbFx_vyQAv_yqNckztTCp49Zdfsr2_D9BbToURPZgcu_VJtLTVGubfdruOBUVN4N2vbq_96WTQ8oT9GRdqPA4c87F8TUrJm1V4Ejp-HfS-f2dcO9vWr08aQDDVUzzwUAL_cF1_YulAMydKQEPV8tvpm5PLfwtfauzHOlkZ_1hBQJf5A%3D%3D&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=999a7175bfaeee0c&q=python+jupyter&l=Tucson&tk=1g0fe4us1hv6q801&from=ja&advn=5550347756246289&adid=387601889&ad=-6NYlbfkN0BeTVAYNccvDEmPph7gZYfEOxyr-xfXmbJSPMpMvIORMf9jythPQ5rpqGth5Fnvc66aC9vOKGHFexzwSwWo88zrWr0EIimSRCPw1YssmqbDt1Bed7uA_t65hcBeTl0j-z90b_ijDZmZt2ISu6ekLP6sbijLCnwUjEEtiNXLyBfTxW7VMmsxH04T8-VGdp0Tz_SSGwXar8GyXykuhKLe78Ut63nDMsglvgIAzFBlWGWHbNidsh29aqbLYfQ75GlZW0ZNoicaBhkzUU_BbKsMJOO8lzw-Q4-EVv7upPI-WmTcf3WqIka2yQuaVfb9FKGn9qOwJ8ZmAlZjE2mLVBJer8cswlCAj1w8qUWOtm6pBybq311Cx7_tWH4CsokiHlzmRX6qaFCtgmP5vg%3D%3D&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=999a7175bfaeee0c&q=python+jupyter&l=Tucson&tk=1g0fe4us1hv6q801&from=ja&advn=5550347756246289&adid=387601889&ad=-6NYlbfkN0BeTVAYNccvDEmPph7gZYfEOxyr-xfXmbJSPMpMvIORMf9jythPQ5rpqGth5Fnvc66aC9vOKGHFexzwSwWo88zrWr0EIimSRCPw1YssmqbDt1Bed7uA_t65hcBeTl0j-z90b_ijDZmZt2ISu6ekLP6sbijLCnwUjEEtiNXLyBfTxW7VMmsxH04T8-VGdp0Tz_SSGwXar8GyXykuhKLe78Ut63nDMsglvgIAzFBlWGWHbNidsh29aqbLYfQ75GlZW0ZNoicaBhkzUU_BbKsMJOO8lzw-Q4-EVv7upPI-WmTcf3WqIka2yQuaVfb9FKGn9qOwJ8ZmAlZjE2mLVBJer8cswlCAj1w8qUWOtm6pBybq311Cx7_tWH4CsokiHlzmRX6qaFCtgmP5vg%3D%3D&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=9b210ac3122de2ef&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7536404725441878&adid=387070476&ad=-6NYlbfkN0Dr_hnQ-Jfvx8x1kl6LZGw5tBy_C31iSfGQgc5_4VANfmnlk7wUsb9qjlaUE66B-57Xn_REJ0k3MQ80EdN4wtbnnhXuD171j_82RQNQ0fzx33BF5VZoP90qWgY87H0d341XbwxDh2KqZ1F5dR5OSckiBw3DuNaDQhd7j0l5logY2dskHjT1-0TiC7XvW2YdS0Ak-kRObAL9tmJpcGJqI6gSnq703AKsJqtPFmYIycc4ioUvfHDMwcNsflwD83zX6uGJWLgEMH54BSerEapRAX5khqP30Obl52ZhiRoYwffakfnYzs71llaADkR8BdcF1nEYgOxXHgGofNL4YKQ11Vje4FQnvyZVdMfnEstWklsGd1_7msGjbfZ2uzkqKk9c65A%3D&sjdu=QWF4TUFyrHvH7u082A4fa5I2fJobu6fRLfo7RTiLEnE&acatk=1g0hp9semkuj3800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=9b210ac3122de2ef&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=7536404725441878&adid=387070476&ad=-6NYlbfkN0Dr_hnQ-Jfvx8x1kl6LZGw5tBy_C31iSfGQgc5_4VANfmnlk7wUsb9qjlaUE66B-57Xn_REJ0k3MQ80EdN4wtbnnhXuD171j_82RQNQ0fzx33BF5VZoP90qWgY87H0d341XbwxDh2KqZ1F5dR5OSckiBw3DuNaDQhd7j0l5logY2dskHjT1-0TiC7XvW2YdS0Ak-kRObAL9tmJpcGJqI6gSnq703AKsJqtPFmYIycc4ioUvfHDMwcNsflwD83zX6uGJWLgEMH54BSerEapRAX5khqP30Obl52ZhiRoYwffakfnYzs71llaADkR8BdcF1nEYgOxXHgGofNL4YKQ11Vje4FQnvyZVdMfnEstWklsGd1_7msGjbfZ2uzkqKk9c65A%3D&sjdu=QWF4TUFyrHvH7u082A4fa5I2fJobu6fRLfo7RTiLEnE&acatk=1g0hp9semkuj3800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=a3cfd81643d99415&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=a3cfd81643d99415&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=a99f5a3249cf28c2&q=data%20scientist%20%24100%2C000&l=Tucson%2C%20AZ&tk=1g02poj43hv6q803&from=ja%2CiaBackPress&alid=623cadd96dd9e35db1d27ced&rgtk=1g02poj43hv6q803&ittk=5KXNETJ5EJ',
    'https://www.indeed.com/viewjob?jk=a99f5a3249cf28c2&q=data+scientist+$100,000&l=Tucson,+AZ&tk=1g02poj43hv6q803&from=ja&alid=623cadd96dd9e35db1d27ced&rgtk=1g02poj43hv6q803&ittk=5KXNETJ5EJ',
    'https://www.indeed.com/viewjob?jk=a99f5a3249cf28c2&q=data+scientist+%24100%2C000&l=Tucson%2C+AZ&tk=1g02poj43hv6q803&from=ja&alid=623cadd96dd9e35db1d27ced&rgtk=1g02poj43hv6q803&ittk=5KXNETJ5EJ',
    'https://www.indeed.com/viewjob?jk=a99f5a3249cf28c2&q=data+scientist+%24100%2C000&l=Tucson%2C+AZ&tk=1g02poj43hv6q803&from=ja&alid=623cadd96dd9e35db1d27ced&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g02poj43hv6q803&ittk=5KXNETJ5EJ',
    'https://www.indeed.com/viewjob?jk=b014caee786384b1&q=python%20jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja%2CiaBackPress&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=b014caee786384b1&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=b014caee786384b1&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=b185e67fffc875bb&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=b185e67fffc875bb&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=ba04d3fd9c1be88e&q=data+scientist+$100,000&l=Tucson&tk=1fv19bniop1h1801&from=ja&advn=9078100093439348&adid=378099455&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUef6Vf7cU-S7SzX9kKVwvJibLaVxkh9P-WwG9Mj3xu_gjan7esDOvVSuH3RewgvSrG9nvhGKg2PgcHWT8uuaT1XJGS2ae67NRNBjcwLSvRr9__wt3-_RIM1kIhP29EbeADc6d3SA07CSj9i9BakJztMqDvwmnymH85jatfwKPstkvVKh6LS11VThRa9aCTTh4wf9WLeTSA5qmE0cHV4e4s-g3cvqSnkOHLkoEGKDdGYXke8lWj_pVwK0YlUMZzETcJqDnkE8YTd2Gf6nS1UYOYI6qW-rDiWDfZOOyzqvUWo0rg%3D%3D&sjdu=LClFkqgtkW6SKo_RfYnE68MkziOJDMuPgd4jONrp9liGK3iuJGGzhrTQD3-hhgGOwavki0PiQaN_vosxuN6sapGXdQt-1XpBb0YD6sL7krjLB6uhaPqYW2RRFRA0AMlg1oUGS7IanaRF0aQlMHIKYYL1A9nKpnKQArD9_ql79Im7nKyqOZxW1h5HkGGrFxTOdfKAmA_bPi7rRSohcBoUCK2WaU6AK-XO-Fz2flMNHlqqpCb4IjsRsAbKjiwOX38UDFYP979W4J0vtdmopAwwpw&acatk=1fv1ctbgqn573800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=ba04d3fd9c1be88e&q=data+scientist+%24100%2C000&l=Tucson&tk=1fv19bniop1h1801&from=ja&advn=9078100093439348&adid=378099455&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUef6Vf7cU-S7SzX9kKVwvJibLaVxkh9P-WwG9Mj3xu_gjan7esDOvVSuH3RewgvSrG9nvhGKg2PgcHWT8uuaT1XJGS2ae67NRNBjcwLSvRr9__wt3-_RIM1kIhP29EbeADc6d3SA07CSj9i9BakJztMqDvwmnymH85jatfwKPstkvVKh6LS11VThRa9aCTTh4wf9WLeTSA5qmE0cHV4e4s-g3cvqSnkOHLkoEGKDdGYXke8lWj_pVwK0YlUMZzETcJqDnkE8YTd2Gf6nS1UYOYI6qW-rDiWDfZOOyzqvUWo0rg%3D%3D&sjdu=LClFkqgtkW6SKo_RfYnE68MkziOJDMuPgd4jONrp9liGK3iuJGGzhrTQD3-hhgGOwavki0PiQaN_vosxuN6sapGXdQt-1XpBb0YD6sL7krjLB6uhaPqYW2RRFRA0AMlg1oUGS7IanaRF0aQlMHIKYYL1A9nKpnKQArD9_ql79Im7nKyqOZxW1h5HkGGrFxTOdfKAmA_bPi7rRSohcBoUCK2WaU6AK-XO-Fz2flMNHlqqpCb4IjsRsAbKjiwOX38UDFYP979W4J0vtdmopAwwpw&acatk=1fv1ctbgqn573800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=ba04d3fd9c1be88e&q=data+scientist+%24100%2C000&l=Tucson&tk=1fv19bniop1h1801&from=ja&advn=9078100093439348&adid=378099455&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUef6Vf7cU-S7SzX9kKVwvJibLaVxkh9P-WwG9Mj3xu_gjan7esDOvVSuH3RewgvSrG9nvhGKg2PgcHWT8uuaT1XJGS2ae67NRNBjcwLSvRr9__wt3-_RIM1kIhP29EbeADc6d3SA07CSj9i9BakJztMqDvwmnymH85jatfwKPstkvVKh6LS11VThRa9aCTTh4wf9WLeTSA5qmE0cHV4e4s-g3cvqSnkOHLkoEGKDdGYXke8lWj_pVwK0YlUMZzETcJqDnkE8YTd2Gf6nS1UYOYI6qW-rDiWDfZOOyzqvUWo0rg%3D%3D&sjdu=LClFkqgtkW6SKo_RfYnE68MkziOJDMuPgd4jONrp9liGK3iuJGGzhrTQD3-hhgGOwavki0PiQaN_vosxuN6sapGXdQt-1XpBb0YD6sL7krjLB6uhaPqYW2RRFRA0AMlg1oUGS7IanaRF0aQlMHIKYYL1A9nKpnKQArD9_ql79Im7nKyqOZxW1h5HkGGrFxTOdfKAmA_bPi7rRSohcBoUCK2WaU6AK-XO-Fz2flMNHlqqpCb4IjsRsAbKjiwOX38UDFYP979W4J0vtdmopAwwpw&acatk=1fv1ctbgqn573800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=ba9cf0f1d011b95c&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=ba9cf0f1d011b95c&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=c2d999c92dcb65b3&from=smart-apply&tk=1fvsvj5gj37rg000',
    'https://www.indeed.com/viewjob?jk=c42873f75b456ecd&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801&ittk=NSNELCOKG7',
    'https://www.indeed.com/viewjob?jk=c42873f75b456ecd&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801&ittk=TCRQBGQTFF',
    'https://www.indeed.com/viewjob?jk=c42873f75b456ecd&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801&ittk=NSNELCOKG7',
    'https://www.indeed.com/viewjob?jk=c42873f75b456ecd&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801&ittk=TCRQBGQTFF',
    'https://www.indeed.com/viewjob?jk=cb55e5295839bcd1&q=data+scientist+$100,000&l=Tucson&tk=1fvtn0hd3hv6q803&from=ja&advn=9767451333976163&adid=383082209&ad=-6NYlbfkN0CiRNM7CVr8YueLFKlzwbFWI0o7IjV438l4sVrvKZ0flpURU_mqoI8EzoVZlktGHY-buehiKjQa6sNC-eMLAfM_Zdd87PjblVtOzdrrXqLOV6yuQhuhU8asmdHEY04TMBvL0pBndxxrtJxYkFAJ2WCMZMMKwp6Dyaa5-tSK154HAaF3zYKi0RqbM9oFDUuTwXtFP1bTh7-q_10tVbTGkIV5LRqeQXs0DA2J5oOo0vaIFs6UPzQQwH0J_lsU2lRhRPIOIowbKNNFbGp_XWIb9t_a4wNibR9B0dqgdGRoukrGVMeqyiFTNHsV-G-hfGJ70lfksIwmG4kq22Wz8sMxpLIg3UHOlqgjLf1tMQT8PvHigZuKW5kNmeZ4azohhgy23Ne5luEFi3HKDA%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjMoHkc0JlvZ4aQm3v0rT2lOd6DvqNZA2A4AP5EPBEDQTHwBRui2tFhVrGnetKo8goAkWztH_VrDVytKs8nClCxc&acatk=1fvu08hiut4vm802&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=cb55e5295839bcd1&q=data+scientist+$100,000&l=Tucson&tk=1g0072g75hv6q801&from=ja&advn=9767451333976163&adid=383082209&ad=-6NYlbfkN0CiRNM7CVr8YueLFKlzwbFWI0o7IjV438l4sVrvKZ0flpURU_mqoI8EzoVZlktGHY-buehiKjQa6sNC-eMLAfM_Zdd87PjblVtOzdrrXqLOV6yuQhuhU8asmdHEY04TMBvL0pBndxxrtMYfONS_hvzcLhHAa6Wm40ACU50kgKkMWREYjXiiBdluJDxAue-k35AUZZM9ajmxkRbyGdRm3QT22843Fy8Nd8bdl-aKy225ZsozsYyb15nivNs171GFYjnfKNFPXQngERq_vaMBmnrygSfUl4ERGwO6VBnCjjw19PGrp-4SCbelVZD4OWXxehGRWrjkPdW0MRLPj48YwiVXOczWYHE_o1pi2-cSyzRlESmBYXyTRytHZhM23vuVhWTRVEfkQkTE0R4coAOwtGvv&sjdu=6ByzYMZLGYUgyrbSdN0cjMoHkc0JlvZ4aQm3v0rT2lOd6DvqNZA2A4AP5EPBEDQTHwBRui2tFhVrGnetKo8goAkWztH_VrDVytKs8nClCxc&acatk=1g009lh98q03u800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=cb55e5295839bcd1&q=data+scientist+%24100%2C000&l=Tucson&tk=1fvtn0hd3hv6q803&from=ja&advn=9767451333976163&adid=383082209&ad=-6NYlbfkN0CiRNM7CVr8YueLFKlzwbFWI0o7IjV438l4sVrvKZ0flpURU_mqoI8EzoVZlktGHY-buehiKjQa6sNC-eMLAfM_Zdd87PjblVtOzdrrXqLOV6yuQhuhU8asmdHEY04TMBvL0pBndxxrtJxYkFAJ2WCMZMMKwp6Dyaa5-tSK154HAaF3zYKi0RqbM9oFDUuTwXtFP1bTh7-q_10tVbTGkIV5LRqeQXs0DA2J5oOo0vaIFs6UPzQQwH0J_lsU2lRhRPIOIowbKNNFbGp_XWIb9t_a4wNibR9B0dqgdGRoukrGVMeqyiFTNHsV-G-hfGJ70lfksIwmG4kq22Wz8sMxpLIg3UHOlqgjLf1tMQT8PvHigZuKW5kNmeZ4azohhgy23Ne5luEFi3HKDA%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjMoHkc0JlvZ4aQm3v0rT2lOd6DvqNZA2A4AP5EPBEDQTHwBRui2tFhVrGnetKo8goAkWztH_VrDVytKs8nClCxc&acatk=1fvu08hiut4vm802&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=cb55e5295839bcd1&q=data+scientist+%24100%2C000&l=Tucson&tk=1fvtn0hd3hv6q803&from=ja&advn=9767451333976163&adid=383082209&ad=-6NYlbfkN0CiRNM7CVr8YueLFKlzwbFWI0o7IjV438l4sVrvKZ0flpURU_mqoI8EzoVZlktGHY-buehiKjQa6sNC-eMLAfM_Zdd87PjblVtOzdrrXqLOV6yuQhuhU8asmdHEY04TMBvL0pBndxxrtJxYkFAJ2WCMZMMKwp6Dyaa5-tSK154HAaF3zYKi0RqbM9oFDUuTwXtFP1bTh7-q_10tVbTGkIV5LRqeQXs0DA2J5oOo0vaIFs6UPzQQwH0J_lsU2lRhRPIOIowbKNNFbGp_XWIb9t_a4wNibR9B0dqgdGRoukrGVMeqyiFTNHsV-G-hfGJ70lfksIwmG4kq22Wz8sMxpLIg3UHOlqgjLf1tMQT8PvHigZuKW5kNmeZ4azohhgy23Ne5luEFi3HKDA%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjMoHkc0JlvZ4aQm3v0rT2lOd6DvqNZA2A4AP5EPBEDQTHwBRui2tFhVrGnetKo8goAkWztH_VrDVytKs8nClCxc&acatk=1fvu08hiut4vm802&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=cb55e5295839bcd1&q=data+scientist+%24100%2C000&l=Tucson&tk=1g0072g75hv6q801&from=ja&advn=9767451333976163&adid=383082209&ad=-6NYlbfkN0CiRNM7CVr8YueLFKlzwbFWI0o7IjV438l4sVrvKZ0flpURU_mqoI8EzoVZlktGHY-buehiKjQa6sNC-eMLAfM_Zdd87PjblVtOzdrrXqLOV6yuQhuhU8asmdHEY04TMBvL0pBndxxrtMYfONS_hvzcLhHAa6Wm40ACU50kgKkMWREYjXiiBdluJDxAue-k35AUZZM9ajmxkRbyGdRm3QT22843Fy8Nd8bdl-aKy225ZsozsYyb15nivNs171GFYjnfKNFPXQngERq_vaMBmnrygSfUl4ERGwO6VBnCjjw19PGrp-4SCbelVZD4OWXxehGRWrjkPdW0MRLPj48YwiVXOczWYHE_o1pi2-cSyzRlESmBYXyTRytHZhM23vuVhWTRVEfkQkTE0R4coAOwtGvv&sjdu=6ByzYMZLGYUgyrbSdN0cjMoHkc0JlvZ4aQm3v0rT2lOd6DvqNZA2A4AP5EPBEDQTHwBRui2tFhVrGnetKo8goAkWztH_VrDVytKs8nClCxc&acatk=1g009lh98q03u800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=cb55e5295839bcd1&q=data+scientist+%24100%2C000&l=Tucson&tk=1g0072g75hv6q801&from=ja&advn=9767451333976163&adid=383082209&ad=-6NYlbfkN0CiRNM7CVr8YueLFKlzwbFWI0o7IjV438l4sVrvKZ0flpURU_mqoI8EzoVZlktGHY-buehiKjQa6sNC-eMLAfM_Zdd87PjblVtOzdrrXqLOV6yuQhuhU8asmdHEY04TMBvL0pBndxxrtMYfONS_hvzcLhHAa6Wm40ACU50kgKkMWREYjXiiBdluJDxAue-k35AUZZM9ajmxkRbyGdRm3QT22843Fy8Nd8bdl-aKy225ZsozsYyb15nivNs171GFYjnfKNFPXQngERq_vaMBmnrygSfUl4ERGwO6VBnCjjw19PGrp-4SCbelVZD4OWXxehGRWrjkPdW0MRLPj48YwiVXOczWYHE_o1pi2-cSyzRlESmBYXyTRytHZhM23vuVhWTRVEfkQkTE0R4coAOwtGvv&sjdu=6ByzYMZLGYUgyrbSdN0cjMoHkc0JlvZ4aQm3v0rT2lOd6DvqNZA2A4AP5EPBEDQTHwBRui2tFhVrGnetKo8goAkWztH_VrDVytKs8nClCxc&acatk=1g009lh98q03u800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=ccc8488abbacc098&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=ccc8488abbacc098&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801',
    'https://www.indeed.com/viewjob?jk=d152ccb985c72831&q=predictive+modeler&l=Remote&tk=1g02jb1i1stu4800&from=ja&alid=624c438eb87569687208359f&rgtk=1g02jb1i1stu4800',
    'https://www.indeed.com/viewjob?jk=d152ccb985c72831&q=predictive+modeler&l=Remote&tk=1g02jb1i1stu4800&from=ja&alid=624c438eb87569687208359f&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g02jb1i1stu4800',
    'https://www.indeed.com/viewjob?jk=db2f5c90a42da249&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=8929702876092160&adid=387489674&ad=-6NYlbfkN0B96V2X-ktcizmBETSpagECMuEmqz18d3bUfhM7kAXLfYdWoem_W2FedPZCZAVidzMybIGTrDdgNTp2TOWHVWWfgp05CqiaJTeykA17xNfISmy1f1eWBLv5C_k0YquLhGnRX0Za0zoxWMUt_t5tingQQKXaI9adZ4vBCU8WI6dUN9TNEjrjtrvjWYfb6r6MEFFumYrr6yL7wa3jLXtfN3zhxwFI4T_lN3o8lB8iArdFSHxYL0h2GoFvoXNvZpOHEntWonzOgA-IlrtHtEGV6T9k4aZAJScuvEov--h2vhmc-N9YPvKLa5N_pUdSVGaHto77mCHO33eodx3qHmE7loe20OAc8LqcKQuSi6RUf5ATOw7gLrOW4lcLuuhr6AInpOUdaleKs0GeF5yIpD8p4fW_&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=db2f5c90a42da249&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&advn=8929702876092160&adid=387489674&ad=-6NYlbfkN0B96V2X-ktcizmBETSpagECMuEmqz18d3bUfhM7kAXLfYdWoem_W2FedPZCZAVidzMybIGTrDdgNTp2TOWHVWWfgp05CqiaJTeykA17xNfISmy1f1eWBLv5C_k0YquLhGnRX0Za0zoxWMUt_t5tingQQKXaI9adZ4vBCU8WI6dUN9TNEjrjtrvjWYfb6r6MEFFumYrr6yL7wa3jLXtfN3zhxwFI4T_lN3o8lB8iArdFSHxYL0h2GoFvoXNvZpOHEntWonzOgA-IlrtHtEGV6T9k4aZAJScuvEov--h2vhmc-N9YPvKLa5N_pUdSVGaHto77mCHO33eodx3qHmE7loe20OAc8LqcKQuSi6RUf5ATOw7gLrOW4lcLuuhr6AInpOUdaleKs0GeF5yIpD8p4fW_&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data%20scientist%20%24100%2C000&l=Tucson&tk=1g081icqchv6q800&from=ja%2CiaBackPress&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDd598visTCG_oQN0F8GSPhX9Tw0aOoyCWn-Yrk3Ygil7jTcfMI_CF3txT26W3QgqjsZv-qPzlMcIMyAz41VKAGQviPgQmIaMDM7XfZ-uoUzm9hW1lhKOC6_KJ-UCCwfvYLTC3PoZ5CST3jzAeoJ7Gk2ms3ZF_nFUxNQpcTT8Ehp30cV8YoffohPhjjJux7ear0DfFbBtSCUasWqFLRIcMPJ94aXw98k0oIb0esS14ANg5tjY9VoRv0RF7K3bl579vmg%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1g083onrot4vm800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data+scientist+$100,000&l=Tucson&tk=1fvtn0hd3hv6q803&from=ja&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDd77-zkgTB_eznHoapSRhQiaClYZtYHc8xAlRR-_vu2cLQNKxzPRX9OUGIIOfRYLWOLXpLF2EcyddszkwR1gYnMC1Zo_tkKBRVSOlrPjp5aRJO9ZtggEfHpTxGICAgziMA58Cp8eB7LvW0pA2oAOTEcbdUvhd5EjewuULny75rxuCrhBFZXjOIH2h4Q_vR4fq9EG-Aij490tiskkoNqUbb3P4oZ-w6MjgPkLUaL5eFrol9grT-QLgA5Cti4ke4HqejQ%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1fvu07u0k3kjv000&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data+scientist+$100,000&l=Tucson&tk=1g0072g75hv6q801&from=ja&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDdxtLJlZFOo15RfbswuxS1C8uwzW7H9H6dXGGZtrq6wnW190ZXaieMISBEpOXiajpeB6Sh2hf1zOrt_cA18-hzRJmX0KF39sjusrep0Gc7nyKeSUwBkFi7zMIFRHQp5UyOHIqh6qEFMRnIJ7Y5HJSnd62dhcif6N9TE01a67LF31jcNToSemfuqJvGqGciHgsv-22HpETL_-3kyEe98FGJANUdbU_aKdZkeXKrn-TgyN44qgPSpVBTT5-Eca0f203LQ%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1g009knbvm6p8800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data+scientist+$100,000&l=Tucson&tk=1g05afavjhv6q800&from=ja&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDdwMZw-OhuVxbhLcxBq8zCb00ES2TvbOUkzOC07yXb5cWVgBveERyuTkn_LNU6NMXaaH6G4MzsSWDRRyBkhiJ7GO3VT3kbn_eXChgZmh7lkDQviP9Z_us4NXMViHAvS_FJsRd_noh32cQBvYsnv1wVYOwm8Gcr0wA_sQTzSM9MCXyWKVbHVDDPl3P6gpsznJcpDpMcwHnrOvqJtwIak3sOZ69WlyZC1ZsC2p1JiYWYNGabNpZnYkeI1ukYzFwYzZXXQ%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1g05c8uait63d800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data+scientist+$100,000&l=Tucson&tk=1g081icqchv6q800&from=ja&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDd598visTCG_oQN0F8GSPhX9Tw0aOoyCWn-Yrk3Ygil7jTcfMI_CF3txT26W3QgqjsZv-qPzlMcIMyAz41VKAGQviPgQmIaMDM7XfZ-uoUzm9hW1lhKOC6_KJ-UCCwfvYLTC3PoZ5CST3jzAeoJ7Gk2ms3ZF_nFUxNQpcTT8Ehp30cV8YoffohPhjjJux7ear0DfFbBtSCUasWqFLRIcMPJ94aXw98k0oIb0esS14ANg5tjY9VoRv0RF7K3bl579vmg%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1g083onrot4vm800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data+scientist+%24100%2C000&l=Tucson&tk=1fvtn0hd3hv6q803&from=ja&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDd77-zkgTB_eznHoapSRhQiaClYZtYHc8xAlRR-_vu2cLQNKxzPRX9OUGIIOfRYLWOLXpLF2EcyddszkwR1gYnMC1Zo_tkKBRVSOlrPjp5aRJO9ZtggEfHpTxGICAgziMA58Cp8eB7LvW0pA2oAOTEcbdUvhd5EjewuULny75rxuCrhBFZXjOIH2h4Q_vR4fq9EG-Aij490tiskkoNqUbb3P4oZ-w6MjgPkLUaL5eFrol9grT-QLgA5Cti4ke4HqejQ%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1fvu07u0k3kjv000&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data+scientist+%24100%2C000&l=Tucson&tk=1fvtn0hd3hv6q803&from=ja&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDd77-zkgTB_eznHoapSRhQiaClYZtYHc8xAlRR-_vu2cLQNKxzPRX9OUGIIOfRYLWOLXpLF2EcyddszkwR1gYnMC1Zo_tkKBRVSOlrPjp5aRJO9ZtggEfHpTxGICAgziMA58Cp8eB7LvW0pA2oAOTEcbdUvhd5EjewuULny75rxuCrhBFZXjOIH2h4Q_vR4fq9EG-Aij490tiskkoNqUbb3P4oZ-w6MjgPkLUaL5eFrol9grT-QLgA5Cti4ke4HqejQ%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1fvu07u0k3kjv000&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data+scientist+%24100%2C000&l=Tucson&tk=1g0072g75hv6q801&from=ja&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDdxtLJlZFOo15RfbswuxS1C8uwzW7H9H6dXGGZtrq6wnW190ZXaieMISBEpOXiajpeB6Sh2hf1zOrt_cA18-hzRJmX0KF39sjusrep0Gc7nyKeSUwBkFi7zMIFRHQp5UyOHIqh6qEFMRnIJ7Y5HJSnd62dhcif6N9TE01a67LF31jcNToSemfuqJvGqGciHgsv-22HpETL_-3kyEe98FGJANUdbU_aKdZkeXKrn-TgyN44qgPSpVBTT5-Eca0f203LQ%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1g009knbvm6p8800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data+scientist+%24100%2C000&l=Tucson&tk=1g0072g75hv6q801&from=ja&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDdxtLJlZFOo15RfbswuxS1C8uwzW7H9H6dXGGZtrq6wnW190ZXaieMISBEpOXiajpeB6Sh2hf1zOrt_cA18-hzRJmX0KF39sjusrep0Gc7nyKeSUwBkFi7zMIFRHQp5UyOHIqh6qEFMRnIJ7Y5HJSnd62dhcif6N9TE01a67LF31jcNToSemfuqJvGqGciHgsv-22HpETL_-3kyEe98FGJANUdbU_aKdZkeXKrn-TgyN44qgPSpVBTT5-Eca0f203LQ%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1g009knbvm6p8800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data+scientist+%24100%2C000&l=Tucson&tk=1g05afavjhv6q800&from=ja&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDdwMZw-OhuVxbhLcxBq8zCb00ES2TvbOUkzOC07yXb5cWVgBveERyuTkn_LNU6NMXaaH6G4MzsSWDRRyBkhiJ7GO3VT3kbn_eXChgZmh7lkDQviP9Z_us4NXMViHAvS_FJsRd_noh32cQBvYsnv1wVYOwm8Gcr0wA_sQTzSM9MCXyWKVbHVDDPl3P6gpsznJcpDpMcwHnrOvqJtwIak3sOZ69WlyZC1ZsC2p1JiYWYNGabNpZnYkeI1ukYzFwYzZXXQ%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1g05c8uait63d800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data+scientist+%24100%2C000&l=Tucson&tk=1g05afavjhv6q800&from=ja&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDdwMZw-OhuVxbhLcxBq8zCb00ES2TvbOUkzOC07yXb5cWVgBveERyuTkn_LNU6NMXaaH6G4MzsSWDRRyBkhiJ7GO3VT3kbn_eXChgZmh7lkDQviP9Z_us4NXMViHAvS_FJsRd_noh32cQBvYsnv1wVYOwm8Gcr0wA_sQTzSM9MCXyWKVbHVDDPl3P6gpsznJcpDpMcwHnrOvqJtwIak3sOZ69WlyZC1ZsC2p1JiYWYNGabNpZnYkeI1ukYzFwYzZXXQ%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1g05c8uait63d800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data+scientist+%24100%2C000&l=Tucson&tk=1g081icqchv6q800&from=ja&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDd598visTCG_oQN0F8GSPhX9Tw0aOoyCWn-Yrk3Ygil7jTcfMI_CF3txT26W3QgqjsZv-qPzlMcIMyAz41VKAGQviPgQmIaMDM7XfZ-uoUzm9hW1lhKOC6_KJ-UCCwfvYLTC3PoZ5CST3jzAeoJ7Gk2ms3ZF_nFUxNQpcTT8Ehp30cV8YoffohPhjjJux7ear0DfFbBtSCUasWqFLRIcMPJ94aXw98k0oIb0esS14ANg5tjY9VoRv0RF7K3bl579vmg%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1g083onrot4vm800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e3712e0083ea3b15&q=data+scientist+%24100%2C000&l=Tucson&tk=1g081icqchv6q800&from=ja&advn=7348584799939584&adid=387296906&ad=-6NYlbfkN0BJV66PMW1vfRBroK9Xo9q-fe_BYhfQZJW05MGq2kEzW4fWSUXccOgShbGFPmRHRxMKS6yhBhjxpvgJiPAV_b30ctjS8r95keOZ3UqMVm9ae45uIs8QG9KhA-MCrs2ZHP9BzTvUBoaDd598visTCG_oQN0F8GSPhX9Tw0aOoyCWn-Yrk3Ygil7jTcfMI_CF3txT26W3QgqjsZv-qPzlMcIMyAz41VKAGQviPgQmIaMDM7XfZ-uoUzm9hW1lhKOC6_KJ-UCCwfvYLTC3PoZ5CST3jzAeoJ7Gk2ms3ZF_nFUxNQpcTT8Ehp30cV8YoffohPhjjJux7ear0DfFbBtSCUasWqFLRIcMPJ94aXw98k0oIb0esS14ANg5tjY9VoRv0RF7K3bl579vmg%3D%3D&sjdu=6ByzYMZLGYUgyrbSdN0cjJJgUcV3GBmJwDYcIXdQOd67N_L8MxSVOQ2KsDhzTQ5cmd0BAeJCOIN3kaMYvdV1KU97wf9BlT3H-Uig3_ioKRt-PCqEUiX3jMUhDBUhtPF3WHVGusKeVHpBlJLKh0GT84pHtTGdFsFZ7J65HNzYdSmeslmz5xwObuwld65l_hX-aUsBFny8vfGnIWuaeuL_KbaKOmjh1fxg_kuIyvX5PH268l0I0njpC_zW-58_uFCIwz7H9DxAbtMLOBIWYt2psftlIm_ZfAe3DNMYbSTAykY0Nm_SJd9fWSAsMFSKnXIb&acatk=1g083onrot4vm800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=e8b07daf18046772&q=data+scientist+$100,000&l=Tucson&tk=1fv91q3m7pijt801&from=ja&advn=9078100093439348&adid=378099455&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUeexrsVOrH16jGl8i2YIeYYVcsQUs5dxsisaQQGXTEAJnliZ8xsC6B1OEm8Ym4Pa4pyUmQ2beZlVWSvii12dVdADixwSivE9-2eZl8xLdQMFQw0-lvn1g1jy4GhpSmITM2wBK9_QW9Jtx8b4IhBGBd_dY3c0ZVRFCtrAypjJy41fxJHiu_pFw0RpEIqBZa4dlFK8VrBrnfUQRQTo_jL00Ju6pNkL5q0ymEjj6YfmWLpwJQ3cdSTjZ6gl4FgD6f3BOOHb3sic8cXUjeNGwB3bS7uNnkZ5VPOTOeySH7_yeaMlOw%3D%3D&sjdu=LClFkqgtkW6SKo_RfYnE68MkziOJDMuPgd4jONrp9lg-IJOsCtKScRuYrfgr4L5kX2hyDeOylXMUp_5khV3sdohxSZ4WDBoi3dcrd3Iox_PMA3ymDR63blfBLxnYXhx0V9TsfpJgvAObB325G_Syxb5NstiX0IgiJa-yN0zEhYCRADREu_ON4h90EHRBhOYH0C10AtPAVwDYipuDLCzaJ0AZ0X8WLtIAliLylj7YKNV66AZ2KGpDY97sim5CMSsxL0eu_luU8Tn6ybLUQXFl6oK-4J6WiwJqCs5Fopx3nGM&acatk=1fv91vpjiu2no800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e8b07daf18046772&q=data+scientist+$100,000&l=Tucson&tk=1fvr6pciihv6q800&from=ja&advn=9078100093439348&adid=378099455&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUeexrsVOrH16jGl8i2YIeYYVcsQUs5dxsisaQQGXTEAJnliZ8xsC6B1OP1QvP-xpyZ4KGGI8vSGrKM7tNrFJYVMod4QEkyxc2-_f90Tll_kdt-BpPRhLETsGykfjj1-X9vU8rnbD5xrzIMvGdcDIW_wp8imv67RTl4lT8EhPa2QHcc2bKCPKcXgxHbWZuWg969ZrMh31wI7wydIioAgfQLo7SqT7_zXzjYBb1W9_0nFNQHrw2zte8jFha1F1oQSGUi5OXZhFqnAJ3XI2JO7G_0WBntBLtsTuP-OFtkNlTHgr6w%3D%3D&sjdu=LClFkqgtkW6SKo_RfYnE68MkziOJDMuPgd4jONrp9lg-IJOsCtKScRuYrfgr4L5kX2hyDeOylXMUp_5khV3sdohxSZ4WDBoi3dcrd3Iox_PMA3ymDR63blfBLxnYXhx0V9TsfpJgvAObB325G_Syxb5NstiX0IgiJa-yN0zEhYCRADREu_ON4h90EHRBhOYH0C10AtPAVwDYipuDLCzaJ0AZ0X8WLtIAliLylj7YKNV66AZ2KGpDY97sim5CMSsxL0eu_luU8Tn6ybLUQXFl6oK-4J6WiwJqCs5Fopx3nGM&acatk=1fvsn77pmkf3l800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e8b07daf18046772&q=data+scientist+%24100%2C000&l=Tucson&tk=1fv91q3m7pijt801&from=ja&advn=9078100093439348&adid=378099455&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUeexrsVOrH16jGl8i2YIeYYVcsQUs5dxsisaQQGXTEAJnliZ8xsC6B1OEm8Ym4Pa4pyUmQ2beZlVWSvii12dVdADixwSivE9-2eZl8xLdQMFQw0-lvn1g1jy4GhpSmITM2wBK9_QW9Jtx8b4IhBGBd_dY3c0ZVRFCtrAypjJy41fxJHiu_pFw0RpEIqBZa4dlFK8VrBrnfUQRQTo_jL00Ju6pNkL5q0ymEjj6YfmWLpwJQ3cdSTjZ6gl4FgD6f3BOOHb3sic8cXUjeNGwB3bS7uNnkZ5VPOTOeySH7_yeaMlOw%3D%3D&sjdu=LClFkqgtkW6SKo_RfYnE68MkziOJDMuPgd4jONrp9lg-IJOsCtKScRuYrfgr4L5kX2hyDeOylXMUp_5khV3sdohxSZ4WDBoi3dcrd3Iox_PMA3ymDR63blfBLxnYXhx0V9TsfpJgvAObB325G_Syxb5NstiX0IgiJa-yN0zEhYCRADREu_ON4h90EHRBhOYH0C10AtPAVwDYipuDLCzaJ0AZ0X8WLtIAliLylj7YKNV66AZ2KGpDY97sim5CMSsxL0eu_luU8Tn6ybLUQXFl6oK-4J6WiwJqCs5Fopx3nGM&acatk=1fv91vpjiu2no800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e8b07daf18046772&q=data+scientist+%24100%2C000&l=Tucson&tk=1fv91q3m7pijt801&from=ja&advn=9078100093439348&adid=378099455&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUeexrsVOrH16jGl8i2YIeYYVcsQUs5dxsisaQQGXTEAJnliZ8xsC6B1OEm8Ym4Pa4pyUmQ2beZlVWSvii12dVdADixwSivE9-2eZl8xLdQMFQw0-lvn1g1jy4GhpSmITM2wBK9_QW9Jtx8b4IhBGBd_dY3c0ZVRFCtrAypjJy41fxJHiu_pFw0RpEIqBZa4dlFK8VrBrnfUQRQTo_jL00Ju6pNkL5q0ymEjj6YfmWLpwJQ3cdSTjZ6gl4FgD6f3BOOHb3sic8cXUjeNGwB3bS7uNnkZ5VPOTOeySH7_yeaMlOw%3D%3D&sjdu=LClFkqgtkW6SKo_RfYnE68MkziOJDMuPgd4jONrp9lg-IJOsCtKScRuYrfgr4L5kX2hyDeOylXMUp_5khV3sdohxSZ4WDBoi3dcrd3Iox_PMA3ymDR63blfBLxnYXhx0V9TsfpJgvAObB325G_Syxb5NstiX0IgiJa-yN0zEhYCRADREu_ON4h90EHRBhOYH0C10AtPAVwDYipuDLCzaJ0AZ0X8WLtIAliLylj7YKNV66AZ2KGpDY97sim5CMSsxL0eu_luU8Tn6ybLUQXFl6oK-4J6WiwJqCs5Fopx3nGM&acatk=1fv91vpjiu2no800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=e8b07daf18046772&q=data+scientist+%24100%2C000&l=Tucson&tk=1fvr6pciihv6q800&from=ja&advn=9078100093439348&adid=378099455&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUeexrsVOrH16jGl8i2YIeYYVcsQUs5dxsisaQQGXTEAJnliZ8xsC6B1OP1QvP-xpyZ4KGGI8vSGrKM7tNrFJYVMod4QEkyxc2-_f90Tll_kdt-BpPRhLETsGykfjj1-X9vU8rnbD5xrzIMvGdcDIW_wp8imv67RTl4lT8EhPa2QHcc2bKCPKcXgxHbWZuWg969ZrMh31wI7wydIioAgfQLo7SqT7_zXzjYBb1W9_0nFNQHrw2zte8jFha1F1oQSGUi5OXZhFqnAJ3XI2JO7G_0WBntBLtsTuP-OFtkNlTHgr6w%3D%3D&sjdu=LClFkqgtkW6SKo_RfYnE68MkziOJDMuPgd4jONrp9lg-IJOsCtKScRuYrfgr4L5kX2hyDeOylXMUp_5khV3sdohxSZ4WDBoi3dcrd3Iox_PMA3ymDR63blfBLxnYXhx0V9TsfpJgvAObB325G_Syxb5NstiX0IgiJa-yN0zEhYCRADREu_ON4h90EHRBhOYH0C10AtPAVwDYipuDLCzaJ0AZ0X8WLtIAliLylj7YKNV66AZ2KGpDY97sim5CMSsxL0eu_luU8Tn6ybLUQXFl6oK-4J6WiwJqCs5Fopx3nGM&acatk=1fvsn77pmkf3l800&pub=0cace3277f6b99df',
    'https://www.indeed.com/viewjob?jk=e8b07daf18046772&q=data+scientist+%24100%2C000&l=Tucson&tk=1fvr6pciihv6q800&from=ja&advn=9078100093439348&adid=378099455&ad=-6NYlbfkN0DPiPBVS3KmjTqjymWZvPzPJEsEe9JFHhSvGO4UmOZP0kOy32CS6_PT7hbGwRraUeexrsVOrH16jGl8i2YIeYYVcsQUs5dxsisaQQGXTEAJnliZ8xsC6B1OP1QvP-xpyZ4KGGI8vSGrKM7tNrFJYVMod4QEkyxc2-_f90Tll_kdt-BpPRhLETsGykfjj1-X9vU8rnbD5xrzIMvGdcDIW_wp8imv67RTl4lT8EhPa2QHcc2bKCPKcXgxHbWZuWg969ZrMh31wI7wydIioAgfQLo7SqT7_zXzjYBb1W9_0nFNQHrw2zte8jFha1F1oQSGUi5OXZhFqnAJ3XI2JO7G_0WBntBLtsTuP-OFtkNlTHgr6w%3D%3D&sjdu=LClFkqgtkW6SKo_RfYnE68MkziOJDMuPgd4jONrp9lg-IJOsCtKScRuYrfgr4L5kX2hyDeOylXMUp_5khV3sdohxSZ4WDBoi3dcrd3Iox_PMA3ymDR63blfBLxnYXhx0V9TsfpJgvAObB325G_Syxb5NstiX0IgiJa-yN0zEhYCRADREu_ON4h90EHRBhOYH0C10AtPAVwDYipuDLCzaJ0AZ0X8WLtIAliLylj7YKNV66AZ2KGpDY97sim5CMSsxL0eu_luU8Tn6ybLUQXFl6oK-4J6WiwJqCs5Fopx3nGM&acatk=1fvsn77pmkf3l800&pub=0cace3277f6b99df&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails',
    'https://www.indeed.com/viewjob?jk=f1d11963129879e0&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801&rx_p=GIYZ7HNJ5A',
    'https://www.indeed.com/viewjob?jk=f1d11963129879e0&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&rgtk=1g0fjeco0tipk801&rx_p=XACGXTANEJ',
    'https://www.indeed.com/viewjob?jk=f1d11963129879e0&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801&rx_p=GIYZ7HNJ5A',
    'https://www.indeed.com/viewjob?jk=f1d11963129879e0&q=python+jupyter&l=Remote&tk=1g0fjeco0tipk801&from=ja&alid=6254377b33b425113af40aa2&utm_campaign=job_alerts&utm_medium=email&utm_source=jobseeker_emails&rgtk=1g0fjeco0tipk801&rx_p=XACGXTANEJ',
    'https://www.indeed.com/viewjob?viewtype=embedded&jk=5927397673ded4b1&from=vjs%2CiaBackPress&tk=1g0ccifoct4tu800&hidecmpheader=0&advn=7499600371635092&adid=361249029&topwindowlocation=%2Fjobs%3Fq%3Dpython%2520jupyter%26l%3Dremote%26vjk%3Df8c298418b14a078%26advn%3D9693263586969321',
    'https://www.indeed.com/viewjob?viewtype=embedded&jk=5927397673ded4b1&from=vjs,iaBackPress&tk=1g0ccifoct4tu800&hidecmpheader=0&advn=7499600371635092&adid=361249029&topwindowlocation=%2Fjobs%3Fq%3Dpython%2520jupyter%26l%3Dremote%26vjk%3Df8c298418b14a078%26advn%3D9693263586969321']

In [4]:

%run ../load_magic/dataframes.py
from html_analysis import ElementAnalysis

ea = ElementAnalysis()
for i, url_str in enumerate(urls_list):
    page_soup = get_page_soup(url_str)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    for div_soup in row_div_list:
        child_strs_list = ea.ha.get_navigable_children(div_soup, [])
    break

In [5]:

ea.find_basic_quals_section(child_strs_list, verbose=False)

NameError: name 'db' is not defined

In [19]:

# <div id='jobDescriptionText' class='jobsearch-jobDescriptionText'>
print([f'ea.{fn}' for fn in dir(ea) if 'dict' in fn.lower()])
print([f'ea.{fn}' for fn in dir(ea) if not fn.startswith('_')])

['ea.__dict__']
['ea.display_basic_requirements', 'ea.display_reqs_from_url', 'ea.find_basic_quals_section', 'ea.get_idx_list', 'ea.ha', 'ea.hc']


In [15]:

print([f'ea.ha.{fn}' for fn in dir(ea.ha) if 'dict' in fn.lower()])
print([f'ea.ha.{fn}' for fn in dir(ea.ha) if not fn.startswith('_')])

['ea.ha.NAVIGABLE_PARENT_IS_HEADER_DICT', 'ea.ha.NAVIGABLE_PARENT_IS_QUAL_DICT', 'ea.ha.__dict__', 'ea.ha.store_true_or_false_dict']
['ea.ha.CHILDLESS_TAGS_LIST', 'ea.ha.CLF_NAME', 'ea.ha.CMAP', 'ea.ha.GT_REGEX', 'ea.ha.HTML_SCANNER_REGEX', 'ea.ha.LT_REGEX', 'ea.ha.NAVIGABLE_PARENT_IS_HEADER_DICT', 'ea.ha.NAVIGABLE_PARENT_IS_QUAL_DICT', 'ea.ha.QUALS_SCANNER_REGEX', 'ea.ha.SAVES_HTML_FOLDER', 'ea.ha.clean_html_str', 'ea.ha.get_body_soup', 'ea.ha.get_child_strs_from_file', 'ea.ha.get_child_tags_list', 'ea.ha.get_is_header_list', 'ea.ha.get_navigable_children', 'ea.ha.html2text', 'ea.ha.html_regex_tokenizer', 'ea.ha.quals_regex_tokenizer', 'ea.ha.store_true_or_false_dict', 'ea.ha.store_unique_list']


In [16]:

print([f'ea.hc.{fn}' for fn in dir(ea.hc) if 'dict' in fn.lower()])
print([f'ea.hc.{fn}' for fn in dir(ea.hc) if not fn.startswith('_')])

['ea.hc.POS_EXPLANATION_DICT', 'ea.hc.__dict__']
['ea.hc.CORP_SCOPE_HEADERS_LIST', 'ea.hc.EDUC_REQS_HEADERS_LIST', 'ea.hc.INTERV_PROC_HEADERS_LIST', 'ea.hc.JOB_DURATION_HEADERS_LIST', 'ea.hc.JOB_TITLE_HEADERS_LIST', 'ea.hc.LEGAL_NOTIFS_HEADERS_LIST', 'ea.hc.OFFICE_LOC_HEADERS_LIST', 'ea.hc.OTHER_HEADERS_LIST', 'ea.hc.POST_DATE_HEADERS_LIST', 'ea.hc.POS_EXPLANATION_DICT', 'ea.hc.PREFF_QUALS_HEADERS_LIST', 'ea.hc.REQ_QUALS_HEADERS_LIST', 'ea.hc.SUPP_PAY_HEADERS_LIST', 'ea.hc.TASK_SCOPE_HEADERS_LIST', 'ea.hc.append_parts_of_speech_list', 'ea.hc.get_feature_tuple']


In [12]:

ea.find_basic_quals_section?

Signature: ea.find_basic_quals_section(child_strs_list, verbose=False)
Docstring: <no docstring>
File:      c:\users\daveb\onedrive\documents\github\job-hunting\py\html_analysis.py
Type:      method


In [13]:

ea.ha.get_child_strs_from_file?

Signature: ea.ha.get_child_strs_from_file(file_name)
Docstring: <no docstring>
File:      c:\users\daveb\onedrive\documents\github\job-hunting\py\html_analysis.py
Type:      method
